In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# %%
import torch
import string

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.5 MB/s 


In [4]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 636 kB 70.9 MB/s 
     |████████████████████████████████| 3.3 MB 79.0 MB/s 
     |████████████████████████████████| 895 kB 78.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [6]:
with open('stockholm/wikipedia_music/verbs/wikipedia_masked_verbs.txt', 'r') as fp:
    text = fp.read().split('\n')

##BERT

In [7]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [9]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [10]:
text = text["sentence"]

In [11]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [12]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [13]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [14]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [15]:
inputs

{'input_ids': tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [16]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [17]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [18]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [19]:
mask_arr

tensor([[False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False,  True,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [20]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [21]:
selection[:5]

[[2, 9], [5, 11, 17], [5], [4, 5], [1, 9, 11]]

In [22]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [23]:
inputs.input_ids

tensor([[  101,  1000,   103,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,   103,   103,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [24]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [25]:
dataset = MeditationsDataset(inputs)

In [26]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [27]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [28]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [29]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [00:54<00:00,  1.66s/it, loss=0.061]


##XLM

In [30]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [31]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [32]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [33]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [34]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [35]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [36]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False]])

In [37]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [38]:
selection[:5]

[[3, 9, 12, 15], [5, 17], [3, 5, 17], [2], [5, 19]]

In [39]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [40]:
inputs.input_ids

tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,    103,      1,      1],
        [     0,     44,   4687,  ...,    103,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]])

In [41]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [42]:
dataset = MeditationsDataset(inputs)

In [43]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [44]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [45]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [46]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [02:33<00:00,  4.65s/it, loss=0.583]


In [47]:
bert_model = model.eval()

In [48]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [49]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [50]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [51]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [52]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [53]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [54]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [55]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True,  True],
        [False,  True, False,  ...,  True,  True, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False, False],
        [False,  True, False,  ..., False, False, False]])

In [56]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [57]:
selection[:5]

[[12, 17], [10, 19, 20], [1, 7, 12, 18, 19], [7, 10, 16], [1, 14, 15, 16]]

In [58]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [59]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,   103,   103],
        [    0,   103,    91,  ...,   103,   103,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,   103,     1,     1],
        [    0,   103,     1,  ...,     1,     1,     1]])

In [60]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [61]:
dataset = MeditationsDataset(inputs)

In [62]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [63]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [64]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [03:05<00:00,  5.62s/it, loss=0.328]


In [65]:
bart_model = bart_model.eval()

##RoBERTa

In [66]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [67]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [68]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [69]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [70]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [71]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [72]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False,  True,  ...,  True,  True, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [73]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [74]:
selection[:5]

[[11, 12, 15], [5], [4, 14, 16], [4, 8, 18], [4, 13, 16]]

In [75]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [76]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113,   103,  ...,   103,   103,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]])

In [77]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [78]:
dataset = MeditationsDataset(inputs)

In [79]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [80]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [81]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:01<00:00,  1.86s/it, loss=2.43]


In [82]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [83]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [84]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [85]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [86]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

In [87]:
top_k = 10

In [88]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [89]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [90]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [91]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'tore\nraced\nroared\nswept\nburned',
 'bert': 'spread\nripped\nswept\nflowed\nburned',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'swept\nburned\ntore\nraged\nspread',
 'xlm': 'runs\nspread\nwent\nsank\nfell',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [92]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'ratt\nfog\nrattled\nshaking\ncracked',
 'bert': 'smashed\nshattered\ncracked\nglowing\nrattled',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'melting\nshattered\nshattering\ncollapsing\nclosing',
 'xlm': 'high\nquiet\nhot\nwide\nraised',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [93]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'protect\nescape\nrelease\nsave\nto',
 'bert': 'keep\nprotect\nsave\npreserve\npound',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'recover\nrelease\nheal\nemerge\nretreat',
 'xlm': 'free\ndefend\nrecover\nrelease\nprotect',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [94]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'boost\nstimulate\nrevive\nlift\nfuel',
 'bert': 'stimulate\nfuel\nboost\ndamage\nrevive',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'boost\nstimulate\nrevive\nspur\ngrow',
 'xlm': 'fuel\nhurt\nincrease\nboost\nhelp',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [95]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'smoke\nair\nsw\nwinds\ndri',
 'bert': 'sits\nspread\nlies\nlands\nground',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'lies\nflows\nmelts\nbinds\npools',
 'xlm': 's\ned\nis\nswimming\nstuck',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia

In [96]:
text

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."',
 '" He communicated his anxieties to the psychiatrist ."',
 '" I \'ll ping my accountant -- April 15 is nearing ."',
 '" harmonize one \'s goals with one \'s abilities ."',
 '" The new teacher tends to steamroller ."',
 '" He sifted the information ."',
 '" square the wood with a file ."',
 '" This hotel can accommodate 250 guests ."',
 '" The tugboat tugged the freighter into the harbor ."',
 '" The immigrants were quickly absorbed into society ."',
 '" A star tops the Christmas Tree ."',
 '" The tanker fueled in Bahrain ."',
 '" a terrible stench saluted our nostrils ."',
 '" The tax cut will boost the economy ."',
 '" The horse was galloping along ."',
 '" She was wearing yellow that day ."',
 '" ping your machine in the office ."'

In [97]:
text_sentence = "Music is  <mask> with a vast range of instruments and vocal techniques"
get_all_predictions(text_sentence)

Music is  <mask> with a vast range of instruments and vocal techniques


{'bart': 'played\nfilled\nwritten\ninfused\nperformed',
 'bert': 'composed\nmixed\ncreated\nplayed\nproduced',
 'electra': 'equipped\nperformed\nmade\nprovided\ncombined',
 'roberta': 'fused\nplayed\ncomposed\nfilled\nrich',
 'xlm': 'made\nassociated\npopular\nserved\nconcerned',
 'xlnet': 'made\nassociated\nwith\nprovided\ndone'}

In [98]:
text_sentence = "The word  <mask> from Greek μουσική (mousike; (art) of the Muses"
get_all_predictions(text_sentence)

The word  <mask> from Greek μουσική (mousike; (art) of the Muses


{'bart': 'derives\n""\ncomes\nμ\nis',
 'bert': 'derives\noriginates\ncomes\noriginated\ncame',
 'electra': 'derives\nderived\nis\noriginates\ncomes',
 'roberta': 'stems\nderives\ncomes\ncame\nis',
 'xlm': 'comes\ncame\ncome\ndiffer\nis',
 'xlnet': 'comes\nis\ncoming\ning\nderives'}

In [99]:
text_sentence = "In its most general form, the activities  <mask> music as an art form or cultural activity"
get_all_predictions(text_sentence)

In its most general form, the activities  <mask> music as an art form or cultural activity


{'bart': 'promote\npromoted\npreserve\nemphasize\nglor',
 'bert': 'promote\nemphasize\ninclude\nof\nembrace',
 'electra': 'of\ndefine\ninclude\nare\ninvolve',
 'roberta': 'portray\nconstitute\nrecognize\npreserve\nembrace',
 'xlm': 'of\ninterpret\nuse\nview\ndebate',
 'xlnet': 'music\nare\nactivities\nas\nis'}

In [100]:
text_sentence = "Ancient Greek and Indian philosophers  <mask> music in two parts: melodies, as tones"
get_all_predictions(text_sentence)

Ancient Greek and Indian philosophers  <mask> music in two parts: melodies, as tones


{'bart': 'wrote\narranged\npreserved\ndivided\ncomposed',
 'bert': 'described\ndivided\nclassified\nregarded\ndefined',
 'electra': 'understood\ndescribed\ncomposed\ndescribe\ntaught',
 'roberta': 'understood\ncomposed\ndivided\nconceived\nsplit',
 'xlm': 'divide\nsplit\nmake\ndiscuss\nshare',
 'xlnet': 's\n”\n’\nof\nStudies'}

In [101]:
text_sentence = "However, 20th-century composer John Cage  <mask> that any sound can be music"
get_all_predictions(text_sentence)

However, 20th-century composer John Cage  <mask> that any sound can be music


{'bart': 'claimed\nbelieved\nproved\nemphasized\nasserted',
 'bert': 'argued\nbelieved\nproved\nstated\nargues',
 'electra': 'believes\nbelieved\nsaid\nnoted\nargues',
 'roberta': 'taught\nargued\nbelieved\nstressed\nproved',
 'xlm': 'decided\nrealized\nclaims\nthought\nfound',
 'xlnet': 'said\nthat\nsays\nsay\nannounced'}

In [102]:
text_sentence = "There are many types of music,  <mask> popular music, traditional music, art music,"
get_all_predictions(text_sentence)

There are many types of music,  <mask> popular music, traditional music, art music,


{'bart': 'including\nclassical\nopera\njazz\nsym',
 'bert': 'including\nparticularly\nlike\nespecially\nnamely',
 'electra': 'including\nfrom\nlike\nparticularly\nespecially',
 'roberta': 'including\nmeaning\nnotably\nspanning\nwhist',
 'xlm': '</s>\nincluding\nthe\nits\nwhether',
 'xlnet': 'including\nlike\nas\nmusic\nare'}

In [103]:
text_sentence = "Music may be played or sung and <mask> live at a rock concert or orchestra performance"
get_all_predictions(text_sentence)

Music may be played or sung and <mask> live at a rock concert or orchestra performance


{'bart': 'recorded\nsung\nperformed\noften\nplayed',
 'bert': 'performed\nplayed\nrecorded\nbroadcast\npresented',
 'electra': 'performed\nplayed\nrecorded\nbroadcast\ncomposed',
 'roberta': 'played\nperformed\ndanced\nsung\ndelivered',
 'xlm': 'may\ncould\nto\nwill\ncan',
 'xlnet': 'played\nperformed\nlive\nrecorded\nsung'}

In [104]:
text_sentence = "cultural activities  <mask> from amateur karaoke singing to playing in an amateur"
get_all_predictions(text_sentence)

cultural activities  <mask> from amateur karaoke singing to playing in an amateur


{'bart': 'range\nranged\nvary\nranging\ncan',
 'bert': 'ranged\nrange\nvaried\nranging\nvary',
 'electra': 'range\nranging\nranged\nvary\ngo',
 'roberta': 'evolved\nrange\nspread\nspan\nspanning',
 'xlm': 'range\ndiffer\nextend\nanywhere\ncome',
 'xlnet': 'ranging\nfrom\nmoving\nes\ndifferent'}

In [105]:
text_sentence = "People may <mask> music as a hobby, like a teen playing cello"
get_all_predictions(text_sentence)

People may <mask> music as a hobby, like a teen playing cello


{'bart': 'play\npursue\nlisten\nsing\nstudy',
 'bert': 'play\nenjoy\npursue\nuse\npractice',
 'electra': 'see\nuse\ntake\nenjoy\nconsider',
 'roberta': 'treat\nregard\nsee\nkeep\ntake',
 'xlm': 'treat\nview\nenjoy\nconsider\nuse',
 'xlnet': 'music\nhave\nbe\nmay\nmake'}

In [106]:
text_sentence = "The music industry includes the individuals who  <mask> new songs and musical pieces (such as songwriters and composers)"
get_all_predictions(text_sentence)

The music industry includes the individuals who  <mask> new songs and musical pieces (such as songwriters and composers)


{'bart': 'write\ncompose\ncreate\nproduce\ndevelop',
 'bert': 'create\nwrite\ncompose\nproduce\ndevelop',
 'electra': 'create\nproduce\nwrite\nplay\ndevelop',
 'roberta': 'produce\nwrite\ncompose\ncreate\ndevelop',
 'xlm': 'produce\nwrite\ncompose\ndevelop\ncreate',
 'xlnet': 'write\nare\nproduce\ncreate\nperform'}

In [107]:
text_sentence = "Classical music is art music produced or  <mask> in the traditions of Western culture"
get_all_predictions(text_sentence)

Classical music is art music produced or  <mask> in the traditions of Western culture


{'bart': 'preserved\nperformed\nrecorded\nwritten\nplayed',
 'bert': 'composed\nperformed\nrecorded\nused\ndeveloped',
 'electra': 'used\ndeveloped\nreproduced\npracticed\nproduced',
 'roberta': 'composed\ndeveloped\npreserved\ncreated\nformed',
 'xlm': 'lived\nplayed\nheard\nbuilt\nliving',
 'xlnet': 'or\nis\nin\nthe\neither'}

In [108]:
text_sentence = "The major periods <mask> the medieval (500–1400), Renaissance (1400–1600), Baroque (1600–1750), Classical (1750–1820), Romantic (1800–1910), Modernist (1890–1975) and Postmodern era"
get_all_predictions(text_sentence)

The major periods <mask> the medieval (500–1400), Renaissance (1400–1600), Baroque (1600–1750), Classical (1750–1820), Romantic (1800–1910), Modernist (1890–1975) and Postmodern era


{'bart': 'were\nare\nof\nin\nduring',
 'bert': 'are\nwere\ninclude\nincluded\ncomprise',
 'electra': 'were\nare\nof\ninclude\nincluded',
 'roberta': 'span\nwere\npierced\npreceded\ncomprise',
 'xlm': 'of\nin\nare\nwere\ninclude',
 'xlnet': 'are\nwere\nof\ninclude\nis'}

In [109]:
text_sentence = "The earliest reference to classical music  <mask> by the Oxford English Dictionary is from about 1829"
get_all_predictions(text_sentence)

The earliest reference to classical music  <mask> by the Oxford English Dictionary is from about 1829


{'bart': 'in\npreserved\npublished\nrecorded\nwritten',
 'bert': 'found\nrecorded\ncited\nindexed\nattested',
 'electra': 'published\nprovided\nmade\nused\ncompiled',
 'roberta': 'kept\nmade\nrecorded\nindexed\nmaintained',
 'xlm': 'reported\nprovided\nmade\nfound\nshared',
 'xlnet': 'music\nin\nhistory\nis\nliterature'}

In [110]:
text_sentence = "estern staff notation is <mask> by composers to indicate to the performer"
get_all_predictions(text_sentence)

estern staff notation is <mask> by composers to indicate to the performer


{'bart': 'often\nused\ncommonly\ntypically\nusually',
 'bert': 'used\nemployed\nutilized\ndevised\ncreated',
 'electra': 'used\nrequired\nperformed\nutilized\nintended',
 'roberta': 'used\ncreated\nkept\nwritten\naltered',
 'xlm': 'used\noffered\nprepared\nexpected\nallowed',
 'xlnet': 'is\nused\nare\nwas\nnot'}

In [111]:
text_sentence = "It <mask> both sacred (religious) and secular music"
get_all_predictions(text_sentence)

It <mask> both sacred (religious) and secular music


{'bart': 'plays\nplayed\ncontains\nincorporates\nincludes',
 'bert': 'played\ncontains\nfeatures\nfeatured\ncontained',
 'electra': 'is\nhas\ncontains\nplays\nprovides',
 'roberta': 'plays\nplayed\nfeatures\ncombines\ncontains',
 'xlm': 'is\ns\noffers\nwas\nhas',
 'xlnet': 'is\nalso\nboth\nit\neither'}

In [112]:
text_sentence = "Electronic music is music that  <mask> electronic musical instruments, digital instruments"
get_all_predictions(text_sentence)

Electronic music is music that  <mask> electronic musical instruments, digital instruments


{'bart': 'is\nuses\ncan\nincorporates\nplays',
 'bert': 'incorporates\ncombines\nuses\nmixes\nutilizes',
 'electra': 'includes\nuses\ncombines\ninvolves\nhas',
 'roberta': 'combines\nuses\nblends\nfeatures\ncomprises',
 'xlm': 'includes\nhas\noffers\nfeatures\nis',
 'xlnet': 'is\nincludes\nuses\nelectronic\nsounds'}

In [113]:
text_sentence = "Pure electronic instruments  <mask> entirely on circuitry- <mask> sound generation"
get_all_predictions(text_sentence)

Pure electronic instruments  <mask> entirely on circuitry- <mask> sound generation


{'bart': 'rely\nrun\ndepend\noperate\nare',
 'bert': 'relied\nrely\nbased\nrelying\noperated',
 'electra': 'rely\ndepend\nfocus\nrelies\nbuilt',
 'roberta': 'rely\ndepend\noperate\nwork\nrun',
 'xlm': 'are\nrun\nwere\nwork\nbuilt',
 'xlnet': 'instruments\ncompletely\nentirely\ntotally\ndevices'}

In [114]:
text_sentence = "Electromechanical instruments may <mask> mechanical parts such as strings, hammers, and electric elements"
get_all_predictions(text_sentence)

Electromechanical instruments may <mask> mechanical parts such as strings, hammers, and electric elements


{'bart': 'incorporate\nhave\ncombine\ncontain\nuse',
 'bert': 'contain\nincorporate\ninclude\nhave\nuse',
 'electra': 'have\ninclude\ncontain\nuse\ncomprise',
 'roberta': 'fracture\nfuse\nrepair\nbind\ntransform',
 'xlm': 'contain\ninclude\nhave\ncombine\nbalance',
 'xlnet': 'be\ninclude\nhave\nuse\ncarry'}

In [115]:
text_sentence = " such devices <mask> the telharmonium, Hammond organ, electric piano and the electric guitar"
get_all_predictions(text_sentence)

 such devices <mask> the telharmonium, Hammond organ, electric piano and the electric guitar


{'bart': 'include\nas\nare\nwere\noften',
 'bert': 'included\nas\nwere\ninclude\ncomprised',
 'electra': 'include\nas\nare\nincluded\ncomprise',
 'roberta': 'include\ncomprise\nare\nmuzzle\nanimate',
 'xlm': 'as\nare\nwere\ninclude\nin',
 'xlnet': 'as\nthe\nare\nin\nincluding'}

In [116]:
text_sentence = "The first electronic musical devices were <mask> at the end of the 19th century"
get_all_predictions(text_sentence)

The first electronic musical devices were <mask> at the end of the 19th century


{'bart': 'invented\nintroduced\nmanufactured\nreleased\ncommercially',
 'bert': 'invented\ndeveloped\ncreated\nintroduced\nmanufactured',
 'electra': 'introduced\ninvented\ndeveloped\nestablished\nlaunched',
 'roberta': 'invented\nmanufactured\nproduced\ncreated\ndeveloped',
 'xlm': 'developed\nbuilt\nproduced\ncreated\nmade',
 'xlnet': 'made\nwere\nreleased\nwas\nuntil'}

In [117]:
text_sentence = "During the 1920s and 1930s, a number of electronic instruments were <mask> "
get_all_predictions(text_sentence)

During the 1920s and 1930s, a number of electronic instruments were <mask> 


{'bart': 'invented\nmanufactured\nmade\nused\nproduced',
 'bert': 'invented\ndeveloped\nmanufactured\nproduced\nmade',
 'electra': 'used\nintroduced\nproduced\ninvented\ndeveloped',
 'roberta': 'manufactured\nproduced\ninvented\ndeveloped\ncreated',
 'xlm': 'developed\nproduced\ncreated\nmade\nused',
 'xlnet': 'a\nin\nwere\nan\nbeing'}

In [118]:
text_sentence = "By the 1940s, magnetic audio tape  <mask> musicians to tape sounds"
get_all_predictions(text_sentence)

By the 1940s, magnetic audio tape  <mask> musicians to tape sounds


{'bart': 'enabled\nallowed\ncould\nwas\ntied',
 'bert': 'enabled\nallowed\nsubjected\nexposed\nencouraged',
 'electra': 'allowed\nenabled\nallows\nrequired\nencouraged',
 'roberta': 'enabled\nallowed\npermitted\nempowered\ntaught',
 'xlm': 'allowed\ns\nenabled\nallows\nhelped',
 'xlnet': 'tape\nrecorder\ncassette\nrecording\nequipment'}

In [119]:
text_sentence = "Musique concrète,  <mask> in Paris in 1948,"
get_all_predictions(text_sentence)

Musique concrète,  <mask> in Paris in 1948,


{'bart': 'published\nperformed\nrevived\nreleased\nrecorded',
 'bert': 'premiered\ncreated\nfounded\ninaugurated\nlaunched',
 'electra': 'opened\npublished\nreleased\nbuilt\nfounded',
 'roberta': 'premiered\nperformed\ncomposed\nstaged\ncreated',
 'xlm': 'lived\narrived\nliving\nraised\nmarried',
 'xlnet': 'made\ndone\nfound\nalready\nhere'}

In [120]:
text_sentence = "Music  <mask> solely from electronic generators was first produced in Germany in 1953"
get_all_predictions(text_sentence)

Music  <mask> solely from electronic generators was first produced in Germany in 1953


{'bart': 'pumped\nplayed\nproduced\ncomposed\nderived',
 'bert': 'generated\nderived\nproduced\ncomposed\ncreated',
 'electra': 'produced\ncomposed\nmade\ngenerated\nderived',
 'roberta': 'generated\ndrawn\nproduced\ncreated\nderived',
 'xlm': 'made\ncreated\ninspired\ndeveloped\nproduced',
 'xlnet': 'Music\ncompletely\nmusic\ntotally\nplaying'}

In [121]:
text_sentence = "In the early 1970s, Moog synthesizers and Japanese drum machines helped <mask> synthesized electronic music"
get_all_predictions(text_sentence)

In the early 1970s, Moog synthesizers and Japanese drum machines helped <mask> synthesized electronic music


{'bart': 'to\ntransform\npopular\nproduce\nrevolution',
 'bert': 'create\nrevive\nspread\nproduce\nform',
 'electra': 'to\nproduce\ncreate\nand\ndevelop',
 'roberta': 'produce\ncreate\nform\nshape\nto',
 'xlm': 'make\nproduce\ncreate\nto\nthe',
 'xlnet': 'made\nmake\ncreate\nsynthesized\ncreated'}

In [122]:
text_sentence = "mass produced digital synthesizers, such as the Yamaha DX7, <mask> popular and MIDI"
get_all_predictions(text_sentence)

mass produced digital synthesizers, such as the Yamaha DX7, <mask> popular and MIDI


{'bart': 'are\nwere\nbecame\nhave\nproved',
 'bert': 'became\nwere\nremained\nproved\ngrew',
 'electra': 'are\nboth\nwere\nthe\nis',
 'roberta': 'were\nair\nare\nwhist\ncirculated',
 'xlm': 'are\nis\nwere\nwas\nbe',
 'xlnet': 'are\na\nis\nof\nwith'}

In [123]:
text_sentence = "A music video is a short film or video, of variable length, that  <mask> a song or album with imagery"
get_all_predictions(text_sentence)

A music video is a short film or video, of variable length, that  <mask> a song or album with imagery


{'bart': 'accompanies\nportrays\ndepicts\ndram\npresents',
 'bert': 'fills\npictures\ncombines\nresembles\nlinks',
 'electra': 'depicts\nshows\nincludes\ncovers\ncombines',
 'roberta': 'wraps\ncombines\nfills\npresents\nillustrates',
 'xlm': 'includes\nprovides\nhas\nis\nfeatures',
 'xlnet': 'a\nan\nis\nidentifies\neither'}

In [124]:
text_sentence = "Tie-ins and merchandising can be <mask> for toys or for food"
get_all_predictions(text_sentence)

Tie-ins and merchandising can be <mask> for toys or for food


{'bart': 'used\npaid\nsubstituted\narranged\ntraded',
 'bert': 'used\ntraded\ndone\npurchased\nsold',
 'electra': 'used\ndone\neither\nmade\nboth',
 'roberta': 'used\nexchanged\narranged\nbought\nexploited',
 'xlm': 'used\nuseful\ngood\nmade\ngreat',
 'xlnet': 'be\neither\nwhether\nfor\nin'}

In [125]:
text_sentence = "Although the origins of the music video date back to musical short films that first  <mask> in the 1920"
get_all_predictions(text_sentence)

Although the origins of the music video date back to musical short films that first  <mask> in the 1920


{'bart': 'appeared\nscreened\naired\nran\nplayed',
 'bert': 'appeared\nemerged\ncirculated\npremiered\naired',
 'electra': 'appeared\naired\nreleased\nbegan\npremiered',
 'roberta': 'circulated\naired\nemerged\npremiered\nappeared',
 'xlm': 'appeared\ndeveloped\nbegan\nmade\nreleased',
 'xlnet': 'appeared\nfirst\nappear\nin\nperiod'}

In [126]:
text_sentence = "Prior to the 1980s, these kinds of videos were  <mask> by various terms"
get_all_predictions(text_sentence)

Prior to the 1980s, these kinds of videos were  <mask> by various terms


{'bart': 'referred\noften\nlabeled\nknown\nidentified',
 'bert': 'known\ncalled\ndescribed\nreferred\ndenoted',
 'electra': 'defined\nmade\ncharacterized\ncreated\nproduced',
 'roberta': 'accompanied\npreceded\ncharacterized\naddressed\nknown',
 'xlm': 'shared\ndescribed\nused\nassociated\ncovered',
 'xlnet': 'were\nagain\noften\nby\nmade'}

In [127]:
text_sentence = "music videos <mask> a wide range of styles and contemporary video"
get_all_predictions(text_sentence)

music videos <mask> a wide range of styles and contemporary video


{'bart': 'showcase\nfeature\nincorporate\nexplore\ncombine',
 'bert': 'represent\nexplore\nin\nexplored\ndisplay',
 'electra': 'offer\nin\nshare\nhave\nuse',
 'roberta': 'span\ncover\nspanning\nshow\nportray',
 'xlm': 'in\nof\nwith\nfrom\nfor',
 'xlnet': 'in\nvideos\nhave\nwith\nrange'}

In [128]:
text_sentence = "Some music videos  <mask> different styles with music, such as animation and live-action"
get_all_predictions(text_sentence)

Some music videos  <mask> different styles with music, such as animation and live-action


{'bart': 'combine\nblend\nfuse\nmix\nincorporate',
 'bert': 'combine\nmix\nblend\ncombined\nfuse',
 'electra': 'use\nincorporate\nhave\nshare\noffer',
 'roberta': 'combine\nfuse\nshow\nblend\nmix',
 'xlm': 'have\noffer\nprovide\nshow\nfeature',
 'xlnet': 'videos\nhave\nare\ndo\nvideo'}

In [129]:
text_sentence = "other music videos may not <mask> any concept, being only a filmed version of the song "
get_all_predictions(text_sentence)

other music videos may not <mask> any concept, being only a filmed version of the song 


{'bart': 'adhere\nhave\nfollow\nportray\ncontain',
 'bert': 'have\nrepresent\ncontain\nfit\npresent',
 'electra': 'have\nmatch\ninclude\nexplain\nbe',
 'roberta': 'have\nconvey\ncapture\nexplain\ncommunicate',
 'xlm': 'have\nshare\ncontain\nexpress\ncarry',
 'xlnet': 'seem\nbe\nhave\nmatter\nof'}

In [130]:
text_sentence = "Some types of folk music may be <mask> world music"
get_all_predictions(text_sentence)

Some types of folk music may be <mask> world music


{'bart': 'compared\ncombined\nplayed\nrelated\naccompanied',
 'bert': 'called\nconsidered\ntermed\nnamed\nfound',
 'electra': 'real\nall\nvirtual\ncalled\nlocal',
 'roberta': 'considered\ncalled\nnew\nin\napproaching',
 'xlm': 'the\na\nin\nconsidered\ninto',
 'xlnet': 'be\nbeing\non\nthe\nof'}

In [131]:
text_sentence = "It has been  <mask> with commercial and classical styles"
get_all_predictions(text_sentence)

It has been  <mask> with commercial and classical styles


{'bart': 'infused\ndesigned\nwritten\ncombined\nblended',
 'bert': 'decorated\nadorned\nrenovated\nenriched\nmixed',
 'electra': 'experimented\nassociated\nmixed\nreplaced\ncombined',
 'roberta': 'filled\ndecorated\nmixed\nloaded\npacked',
 'xlm': 'associated\npopular\nshared\ncompared\nalong',
 'xlnet': 'had\nbeen\ndone\nmade\nwith'}

In [132]:
text_sentence = "This type of folk music also  <mask> fusion genres such as folk rock, folk metal, and others"
get_all_predictions(text_sentence)

This type of folk music also  <mask> fusion genres such as folk rock, folk metal, and others


{'bart': 'lends\nblends\ninspires\ninspired\ninfluenced',
 'bert': 'incorporates\nincludes\nencompasses\nincluded\ninspired',
 'electra': 'includes\ncontains\nfeatures\ninvolves\nincorporates',
 'roberta': 'encompasses\ninfluences\nbinds\nforms\ncomprises',
 'xlm': 'has\nincludes\noffers\nprovides\nallows',
 'xlnet': 'also\nfollows\nmore\ndoes\noften'}

In [133]:
text_sentence = "While contemporary folk music <mask> a genre generally distinct from traditional folk music"
get_all_predictions(text_sentence)

While contemporary folk music <mask> a genre generally distinct from traditional folk music


{'bart': 'is\nhas\nexists\nwas\nemerged',
 'bert': 'is\nremains\nwas\nconstitutes\nconstituted',
 'electra': 'is\nrepresents\nremains\nwas\nprovides',
 'roberta': 'is\nforms\ndefines\npresents\noccupies',
 'xlm': 'is\nas\nwas\nin\nhas',
 'xlnet': 'is\na\nmusic\nhas\nare'}

In [134]:
text_sentence = "it often  <mask> the same performers and venues as traditional folk music"
get_all_predictions(text_sentence)

it often  <mask> the same performers and venues as traditional folk music


{'bart': 'played\nplays\nuses\nused\nfeatures',
 'bert': 'featured\nattracted\nfeatures\nused\nhad',
 'electra': 'features\nhas\nuses\nemploys\nincludes',
 'roberta': 'features\nattracts\ndraws\nhas\nuses',
 'xlm': 'has\noffers\nprovides\nincludes\nhad',
 'xlnet': 'the\noften\nenough\nas\nabout'}

In [135]:
text_sentence = " music can be  <mask> into genres in  <mask> ways, such as popular music and art music, or religious music and secular music"
get_all_predictions(text_sentence)

 music can be  <mask> into genres in  <mask> ways, such as popular music and art music, or religious music and secular music


{'bart': 'divided\ncategorized\ngrouped\nbroken\nclassified',
 'bert': 'divided\nclassified\ncategorized\nsplit\nseparated',
 'electra': 'divided\nsubdivided\ncategorized\ntranslated\nintegrated',
 'roberta': 'divided\nsplit\nbroken\nseparated\ngrouped',
 'xlm': 'split\nbroken\ndivide\nspread\ncut',
 'xlnet': 'divided\nbroken\nthrown\nput\nclassified'}

In [136]:
text_sentence = "R&B chart in the 1960s, and many recordings <mask> over into the pop charts"
get_all_predictions(text_sentence)

R&B chart in the 1960s, and many recordings <mask> over into the pop charts


{'bart': 'crossed\nspilled\ndrifted\nwere\nmade',
 'bert': 'crossed\nspilled\ncarried\ncross\npassed',
 'electra': 'came\nspilled\nrolled\nwent\nmoved',
 'roberta': 'crossed\nspilled\ncross\nrolled\ncarried',
 'xlm': 'went\ncame\nturn\nspread\nmoved',
 'xlnet': 'over\nout\nwell\nback\ntop'}

In [137]:
text_sentence = "Britain and elsewhere,  <mask> by the Japanese conglomerate Sony Group Corporation, it is part of the Sony Music Group"
get_all_predictions(text_sentence)

Britain and elsewhere,  <mask> by the Japanese conglomerate Sony Group Corporation, it is part of the Sony Music Group


{'bart': 'it\nSony\nowned\nsupported\nbacked',
 'bert': 'owned\ncontrolled\nrun\nadministered\noperated',
 'electra': 'owned\noperated\nrun\ncontrolled\nfunded',
 'roberta': 'owned\nadministered\ncontrolled\nabsorbed\nmanufactured',
 'xlm': 'run\nproduced\nserved\nled\ndeveloped',
 'xlnet': 'and\nowned\ncontrolled\noperated\nproduced'}

In [138]:
text_sentence = "It was originally  <mask> in 1929 as american record corporation"
get_all_predictions(text_sentence)

It was originally  <mask> in 1929 as american record corporation


{'bart': 'released\nissued\ndistributed\nrecorded\npublished',
 'bert': 'founded\nestablished\nformed\nincorporated\nlaunched',
 'electra': 'established\nincorporated\nknown\nfounded\nformed',
 'roberta': 'formed\nestablished\nregistered\ncreated\nfounded',
 'xlm': 'established\ncreated\nannounced\nreported\nbuilt',
 'xlnet': 'in\nlater\noriginally\nfirst\nreleased'}

In [139]:
text_sentence = "The merger was  <mask> on August 1, 2019"
get_all_predictions(text_sentence)

The merger was  <mask> on August 1, 2019


{'bart': 'finalized\neffective\ncompleted\napproved\nofficially',
 'bert': 'completed\nannounced\nfinalized\napproved\nclosed',
 'electra': 'announced\nlaunched\nconcluded\ncompleted\nfinalized',
 'roberta': 'completed\nfinalized\nsealed\napproved\nsigned',
 'xlm': 'announced\nreported\ncompleted\ndecided\nstarted',
 'xlnet': 'was\non\nagain\nthen\nmade'}

In [140]:
text_sentence = "Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies "
get_all_predictions(text_sentence)

Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies 


{'bart': 'consists\nsings\nincludes\nconsist\ncontains',
 'bert': 'consisted\nconsists\nconsist\nconsisting\ncomposed',
 'electra': 'consists\nspeaks\nconsist\nconsisted\ntalks',
 'roberta': 'consists\ncomprises\nconsisted\nconsist\ncomposed',
 'xlm': 'consist\nmade\nlots\nheard\nspoke',
 'xlnet': 'often\nconsists\nprimarily\nmostly\nconsisting'}

In [141]:
text_sentence = "It  <mask> more than three million album entries and 30 million tracks"
get_all_predictions(text_sentence)

It  <mask> more than three million album entries and 30 million tracks


{'bart': 'sold\nhas\nhad\nsells\ngenerated',
 'bert': 'accumulated\nhad\namassed\nreceived\ncontained',
 'electra': 'has\nhad\nincluded\nincludes\ncontains',
 'roberta': 'accumulated\namassed\nhad\nhas\nsold',
 'xlm': 'has\ns\nhad\nis\noffers',
 'xlnet': 'more\nhas\nis\nmakes\nthem'}

In [142]:
text_sentence = "<mask> in 1991, the database was first made available on the Internet in 1994'"
get_all_predictions(text_sentence)

<mask> in 1991, the database was first made available on the Internet in 1994'


{'bart': '"\'\n</s>\nThe\n<s>\n\'"',
 'bert': 'beginning\nfounded\nestablished\ncreated\nbegun',
 'electra': 'and\nbuilt\nback\nfounded\nstarting',
 'roberta': 'pan\nMet\nreed\nJoined\nMerit',
 'xlm': 'מיטוואך\nPublished\nMeet\nמאנטאג\nAs',
 'xlnet': 'in\nagain\ncopyright\nrenewed\ned'}

In [143]:
text_sentence = "AllMusic is  <mask> by RhythmOne"
get_all_predictions(text_sentence)

AllMusic is  <mask> by RhythmOne


{'bart': 'distributed\npublished\nendorsed\nsubt\nlicensed',
 'bert': 'compiled\nrun\nprovided\ndirected\nsupplied',
 'electra': 'produced\ndirected\nwritten\nedited\nreviewed',
 'roberta': 'administered\nprogrammed\nowned\nmaintained\npowered',
 'xlm': 'provided\nproduced\nwritten\ncovered\nprepared',
 'xlnet': 'is\nwas\nmade\nprovided\ndone'}

In [144]:
text_sentence = "Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum "
get_all_predictions(text_sentence)

Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum 


{'bart': 'written\ncomposed\nsung\nperformed\nrecorded',
 'bert': 'written\ncomposed\nrecorded\npenned\nperformed',
 'electra': 'written\ncomposed\nsung\nperformed\ninspired',
 'roberta': 'composed\nwritten\npenned\ncreated\nconceived',
 'xlm': 'written\nproduced\ninspired\nsung\ncovered',
 'xlnet': 'by\ndirected\nservice\nstarted\nplan'}

In [145]:
text_sentence = "EDM producers also  <mask> their music live in a concert"
get_all_predictions(text_sentence)

EDM producers also  <mask> their music live in a concert


{'bart': 'play\nperform\noften\nplayed\nbroadcast',
 'bert': 'perform\nperformed\nbroadcast\nstreamed\nplayed',
 'electra': 'play\nrecorded\nput\nperform\nplayed',
 'roberta': 'played\nplay\nperformed\nperform\nsang',
 'xlm': 'make\nput\nhave\nmade\nenjoy',
 'xlnet': 'also\ntheir\nmade\nmake\nrecorded'}

In [146]:
text_sentence = "UMG\'s corporate headquarters are <mask> in Hilversum, Netherlands"
get_all_predictions(text_sentence)

UMG's corporate headquarters are <mask> in Hilversum, Netherlands


{'bart': 'located\nbased\nnow\nsituated\ncurrently',
 'bert': 'located\nsituated\nbased\nheadquartered\nstationed',
 'electra': 'located\nbased\nheadquartered\nsituated\ncentered',
 'roberta': 'located\nsituated\nalso\nspread\nnow',
 'xlm': 'located\nestablished\nbuilt\nset\nright',
 'xlnet': 'in\nare\nat\nlocated\nbased'}

In [147]:
text_sentence = "UMG has  <mask> licensing agreements with more than 400 platforms worldwide"
get_all_predictions(text_sentence)

UMG has  <mask> licensing agreements with more than 400 platforms worldwide


{'bart': 'signed\nexclusive\nlicensing\nstruck\nnumerous',
 'bert': 'signed\nlicensed\nlicensing\nbilateral\ndeveloped',
 'electra': 'signed\ndeveloped\nreached\nestablished\nhad',
 'roberta': 'signed\nactive\nbinding\nexclusive\nsecured',
 'xlm': 'reached\nsigned\nwritten\nhad\ndeveloped',
 'xlnet': 'has\nhad\nhave\ninternational\nglobal'}

In [148]:
text_sentence = "Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century"
get_all_predictions(text_sentence)

Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century


{'bart': 'traced\ndated\nattributed\nasc\ntrace',
 'bert': 'traced\ndated\ndate\ndates\nattributed',
 'electra': 'traced\ndated\nattributed\nused\nadapted',
 'roberta': 'traced\ndated\nattributed\ntrace\nextended',
 'xlm': 'compared\nback\nbrought\ncarried\nrelated',
 'xlnet': 'be\nback\nfound\ncan\ndated'}

In [149]:
text_sentence = "Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment"
get_all_predictions(text_sentence)

Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment


{'bart': 'rely\nrelied\ndepend\ndepended\ninsisted',
 'bert': 'relied\nfocused\nsettled\nconcentrated\ncentered',
 'electra': 'rely\nfocus\ndepend\nrelied\ninsist',
 'roberta': 'relied\ndepended\ninsisted\nkept\ncarried',
 'xlm': 'are\nwere\nhave\nis\ngo',
 'xlnet': 'are\non\nup\nchurches\nis'}

In [150]:
text_sentence = "Most of the singing was <mask> a cappella "
get_all_predictions(text_sentence)

Most of the singing was <mask> a cappella 


{'bart': 'sung\ndone\nperformed\nin\neither',
 'bert': 'sung\nconducted\nperformed\nmostly\ndone',
 'electra': 'in\non\nmade\njust\nfrom',
 'roberta': 'in\nsung\nsolo\npitched\nSolo',
 'xlm': 'on\nin\nby\nlike\nconsidered',
 'xlnet': 'a\nin\nan\nwas\njust'}

In [151]:
text_sentence = "The first  <mask> use of the term gospel song probably  <mask> in 1874"
get_all_predictions(text_sentence)

The first  <mask> use of the term gospel song probably  <mask> in 1874


{'bart': 'recorded\nknown\nformal\npublished\nwidespread',
 'bert': 'recorded\ndocumented\nsystematic\nknown\nwritten',
 'electra': 'known\npopular\ncommon\nrecorded\npublic',
 'roberta': 'formal\nrecorded\nwidespread\nlegal\nproper',
 'xlm': 'official\npublic\ncommon\nknown\nsuccessful',
 'xlnet': 'hit\ntime\nsight\never\nin'}

In [152]:
text_sentence = "The original gospel songs were <mask> and composed by authors such as George F"
get_all_predictions(text_sentence)

The original gospel songs were <mask> and composed by authors such as George F


{'bart': 'sung\nwritten\nrecorded\noften\narranged',
 'bert': 'written\ntranscribed\ncollected\ncomposed\nsung',
 'electra': 'written\ncomposed\nrecorded\nsung\nproduced',
 'roberta': 'written\ncomposed\nsung\nconceived\nresearched',
 'xlm': 'written\nplanned\nprepared\ndeveloped\nproduced',
 'xlnet': 'and\nwritten\nmade\ncomposed\nwas'}

In [153]:
text_sentence = "Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet"
get_all_predictions(text_sentence)

Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet


{'bart': 'singer\nsingers\nmusic\nsinging\nmusicians',
 'bert': 'choir\nquartet\nband\nchorus\nmusic',
 'electra': 'includes\nhas\nis\nthe\nprovides',
 'roberta': 'fused\npresents\nspread\nfeatures\nwith',
 'xlm': 'for\nto\nbenefits\nwith\nof',
 'xlnet': 'all\ngospel\nchurch\nAll\nthe'}

In [154]:
text_sentence = "British black gospel  <mask> to Gospel music of the African diaspora"
get_all_predictions(text_sentence)

British black gospel  <mask> to Gospel music of the African diaspora


{'bart': 'music\nmusicians\nsingers\nconcerts\nartists',
 'bert': 'refers\nreferred\nrelates\nrefer\napplied',
 'electra': 'music\nrefers\ncontributed\nconnections\ncontributions',
 'roberta': 'fused\nspread\ndanced\nadded\ndances',
 'xlm': 's\nmusic\ncompared\nhas\nband',
 'xlnet': 'to\ngospel\nand\nthe\nwith'}

In [155]:
text_sentence = "Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979"
get_all_predictions(text_sentence)

Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979


{'bart': 'released\nrecorded\nissued\ncommercially\npublished',
 'bert': 'released\nissued\nrecorded\nlaunched\ncompleted',
 'electra': 'released\nrecorded\npublished\nannounced\nreissued',
 'roberta': 'released\nrecorded\nissued\ncompleted\npublished',
 'xlm': 'released\npublished\nproduced\nwritten\ncompleted',
 'xlnet': 'released\nmade\nissued\npublished\non'}

In [156]:
text_sentence = "The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999"
get_all_predictions(text_sentence)

The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999


{'bart': 'formed\nfounded\nbased\noriginally\nhaving',
 'bert': 'formed\nfounded\nforming\nbased\nestablished',
 'electra': 'formed\nfounded\nestablished\nreleased\nstarted',
 'roberta': 'formed\nfounded\nforming\nborn\nrecorded',
 'xlm': 'established\nbegan\nbased\nstarted\nborn',
 'xlnet': 'performed\nmade\nappeared\ndescribed\nand'}

In [157]:
text_sentence = "YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google"
get_all_predictions(text_sentence)

YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google


{'bart': 'offered\noperated\nowned\nrun\ndeveloped',
 'bert': 'operated\nprovided\noffered\nowned\npowered',
 'electra': 'provided\noffered\nowned\noperated\nrun',
 'roberta': 'owned\nhosted\nlaunched\ncreated\ndeveloped',
 'xlm': 'offered\nprovided\nused\ndeveloped\nrun',
 'xlnet': 'provided\nservice\noffered\nmade\nserved'}

In [158]:
text_sentence = "It  <mask> a  tailored interface for the service"
get_all_predictions(text_sentence)

It  <mask> a  tailored interface for the service


{'bart': 'provides\ndesigned\ncreates\nhas\nalso',
 'bert': 'was\nprovided\noffered\ncreated\nhad',
 'electra': 'provides\nprovided\nis\nwas\nhas',
 'roberta': 'created\nwas\nformed\nmade\nbuilt',
 'xlm': 'is\nhas\nprovides\noffers\nwas',
 'xlnet': 'it\nis\nIt\na\nin'}

In [159]:
text_sentence = "The service also  <mask> a premium tier"
get_all_predictions(text_sentence)

The service also  <mask> a premium tier


{'bart': 'has\nincludes\ncomes\noffers\noffered',
 'bert': 'offers\noffered\nadded\nfeatures\nintroduced',
 'electra': 'has\noffers\nfeatures\nincludes\nprovides',
 'roberta': 'has\noffers\ncarries\nfeatures\noffered',
 'xlm': 'has\nprovides\noffers\nhad\nis',
 'xlnet': 'also\nas\nstill\nwith\non'}

In [160]:
text_sentence = "These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium"
get_all_predictions(text_sentence)

These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium


{'bart': 'available\noffered\nextended\naccessible\ncomplimentary',
 'bert': 'available\noffered\nprovided\nextended\ndistributed',
 'electra': 'available\nprovided\naccessible\nextended\ngiven',
 'roberta': 'available\nextended\nafforded\noffered\ngranted',
 'xlm': 'available\noffered\nlimited\nadded\napplied',
 'xlnet': 'to\nnot\nonly\nalso\navailable'}

In [161]:
text_sentence = "In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands"
get_all_predictions(text_sentence)

In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands


{'bart': 'often\nregularly\nreferred\nlistened\ncommonly',
 'bert': 'referred\ncompared\nlistened\nlikened\ndrawn',
 'electra': 'referred\nlinked\npromoted\nsigned\nopened',
 'roberta': 'listened\nreferred\nsigned\nattached\nadded',
 'xlm': 'brought\nreported\ncompared\nadded\nconnected',
 'xlnet': 'referred\nalso\nto\ninitially\nall'}

In [162]:
text_sentence = "It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra"
get_all_predictions(text_sentence)

It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra


{'bart': 'featured\nstarred\nboasted\nhad\nwas',
 'bert': 'featured\nincluded\nstarred\nhosted\nhad',
 'electra': 'features\nhas\nincludes\nfeatured\nstars',
 'roberta': 'drew\nattracted\nfeatures\nhad\nsaw',
 'xlm': 'has\ns\nfeatures\nincludes\nhad',
 'xlnet': 'guest\nis\nin\nit\nhas'}

In [163]:
text_sentence = "Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records"
get_all_predictions(text_sentence)

Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records


{'bart': 'released\nissued\nrecorded\npublished\ndistributed',
 'bert': 'released\nissued\nrecorded\ndistributed\nproduced',
 'electra': 'released\nreissued\nrecorded\npublished\nsold',
 'roberta': 'released\nissued\npressed\nrecorded\npublished',
 'xlm': 'released\nproduced\npublished\nannounced\nwritten',
 'xlnet': 'released\nproduced\nrecorded\nmade\nissued'}

In [164]:
text_sentence = "It  <mask> on Billboard\\'s Top LP\\'s & Tapes chart that same month"
get_all_predictions(text_sentence)

It  <mask> on Billboard\'s Top LP\'s & Tapes chart that same month


{'bart': 'debuted\nwas\npeaked\nchart\nlanded',
 'bert': 'appeared\ndebuted\nlanded\nwas\ncharted',
 'electra': 'appeared\ndebuted\nwas\npremiered\nwent',
 'roberta': 'landed\nappeared\ndebuted\nwas\nwent',
 'xlm': 'appeared\nwas\nappears\nis\ncame',
 'xlnet': '”\nOn\nIt\nchart\nThere'}

In [165]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [166]:
text_sentence = "On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD"
get_all_predictions(text_sentence)

On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD


{'bart': 'released\nissued\npublished\navailable\ndistributed',
 'bert': 'reissued\nreleased\nissued\ndistributed\nfeatured',
 'electra': 'released\nrecorded\nreissued\npublished\nremastered',
 'roberta': 'released\npressed\navailable\nissued\nrecorded',
 'xlm': 'reported\nheard\nreleased\nwritten\nseen',
 'xlnet': 'released\npresented\nannounced\navailable\nalso'}

In [167]:
text_sentence = "Music Choice is  <mask> nationwide by  \u202fComcast Xfinity,\u202fCharter Spectrum,\u202fCox Communications,\u202fVerizon Fios,\u202fDirecTV\u202fand many other mid-size and smaller  MVPDs"
get_all_predictions(text_sentence)

Music Choice is  <mask> nationwide by   Comcast Xfinity, Charter Spectrum, Cox Communications, Verizon Fios, DirecTV and many other mid-size and smaller  MVPDs


{'bart': 'carried\nnow\noffered\ndistributed\navailable',
 'bert': 'distributed\noffered\nprovided\nused\ncarried',
 'electra': 'accessed\noffered\navailable\ndistributed\nprovided',
 'roberta': 'carried\nadministered\nshared\nspread\ndelivered',
 'xlm': 'reported\nbroadcast\nshared\nserved\noffered',
 'xlnet': 'provided\nalso\ns\nis\nmade'}

In [168]:
text_sentence = "music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation"
get_all_predictions(text_sentence)

music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation


{'bart': 'issued\nlicensed\nassigned\napproved\nregistered',
 'bert': 'created\nestablished\nlaunched\nformed\nfounded',
 'electra': 'established\ncreated\ndeveloped\nintroduced\nadopted',
 'roberta': 'created\nregistered\nestablished\nformed\nlaunched',
 'xlm': 'established\ncreated\ndeveloped\nborn\nbuilt',
 'xlnet': 'created\nused\nnamed\nlaunched\nmade'}

In [169]:
text_sentence = "Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)"
get_all_predictions(text_sentence)

Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)


{'bart': 'liability\ncompanies\ncompany\nInterest\nclaim',
 'bert': 'now\nwas\nevidence\nproof\nknowledge',
 'electra': 'notice\nstated\ninformation\nclaims\nnote',
 'roberta': 'Spectrum\nproof\nair\nmention\nshow',
 'xlm': 'satisfied\nrealized\nreported\ndisappointed\nbelief',
 'xlnet': 'Limited\nand\nsaid\nstated\nin'}

In [170]:
text_sentence = "K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture"
get_all_predictions(text_sentence)

K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture


{'bart': 'popular\nthat\nwidely\ncommonly\noften',
 'bert': 'popular\noriginating\noriginated\nproduced\ncreated',
 'electra': 'recorded\nproduced\npopular\nused\nplayed',
 'roberta': 'popular\ncelebrated\npracticed\nspread\ndeveloped',
 'xlm': 'living\npopular\nfound\nproduced\nheard',
 'xlnet': 'popular\nplayed\nperformed\nused\nin'}

In [171]:
text_sentence = "The term K-pop  <mask> popular in the 2000s"
get_all_predictions(text_sentence)

The term K-pop  <mask> popular in the 2000s


{'bart': 'became\nwas\ngrew\nhas\nemerged',
 'bert': 'became\nwas\ngrew\nremained\nbecome',
 'electra': 'became\nwas\ngot\ngrew\nproved',
 'roberta': 'was\nbecame\ngrew\nwent\nproved',
 'xlm': 'became\nwas\nis\nvery\nproved',
 'xlnet': 'pop\npopular\npop\na\nmore'}

In [172]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [173]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [174]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [175]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [176]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'hailed\ndescribed\nacclaimed\nselected\ncited',
 'bert': 'released\nselected\ndescribed\nlaunched\nrecorded',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'chosen\nselected\npurchased\nreleased\nbought',
 'xlm': 'written\nreleased\nproduced\ncreated\nmade',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [177]:
text_sentence = " It provides a tailored interface for the service, oriented towards music streaming, allowing users to <mask> through songs and music videos on YouTube based on genres, playlists, and recommendations"
get_all_predictions(text_sentence)

 It provides a tailored interface for the service, oriented towards music streaming, allowing users to <mask> through songs and music videos on YouTube based on genres, playlists, and recommendations


{'bart': 'browse\nquickly\nsearch\nscroll\nnavigate',
 'bert': 'scroll\nnavigate\nsearch\nfilter\nstream',
 'electra': 'search\nnavigate\ngo\ndownload\nclick',
 'roberta': 'browse\nscroll\nflip\ncomb\nsort',
 'xlm': 'filter\nsearch\nscroll\nread\nclick',
 'xlnet': 'browse\nnavigate\ngo\nplay\nscroll'}

In [178]:
text_sentence = "Other characteristics <mask> a call and response between the lead vocalist and the chorus and an especially tense vocal sound"
get_all_predictions(text_sentence)

Other characteristics <mask> a call and response between the lead vocalist and the chorus and an especially tense vocal sound


{'bart': 'of\ninclude\nwere\nare\nincluded',
 'bert': 'include\nare\nwere\nincluded\nof',
 'electra': 'include\nare\nincluded\ninvolve\nwere',
 'roberta': 'include\nincluded\nshow\nwere\nindicate',
 'xlm': 'include\nare\nof\nwere\nincluded',
 'xlnet': 'are\na\nis\nbetween\nin'}

In [179]:
text_sentence = "Hip hop music, also known as rap music, is a genre of popular music <mask> in the United States by inner-city African Americans and Latino Americans in the Bronx borough of New York City in the 1970s"
get_all_predictions(text_sentence)

Hip hop music, also known as rap music, is a genre of popular music <mask> in the United States by inner-city African Americans and Latino Americans in the Bronx borough of New York City in the 1970s


{'bart': 'popular\nthat\ndeveloped\nproduced\noriginated',
 'bert': 'created\npopular\nformed\noriginated\ncomposed',
 'electra': 'popularized\nintroduced\nproduced\ndeveloped\nreleased',
 'roberta': 'formed\ncreated\ncomposed\ndeveloped\npracticed',
 'xlm': 'heard\nplayed\nproduced\nmade\nshared',
 'xlnet': 'created\nproduced\nstarted\nperformed\ninvented'}

In [180]:
text_sentence = "It consists of a stylized rhythmic music that commonly <mask> rapping, a rhythmic and rhyming speech that is chanted"
get_all_predictions(text_sentence)

It consists of a stylized rhythmic music that commonly <mask> rapping, a rhythmic and rhyming speech that is chanted


{'bart': 'accompanies\nresembles\nincorporates\nsounds\ncan',
 'bert': 'resembles\ninvolves\nincludes\nincorporates\nencompasses',
 'electra': 'involves\nmeans\nincludes\nuses\ndescribes',
 'roberta': 'accompanies\nfeatures\nrequires\nstimulates\nattracts',
 'xlm': 'is\nincludes\nhas\nappears\nwas',
 'xlnet': 'is\nwas\nruns\nbe\nincludes'}

In [181]:
text_sentence = "Rapping <mask> as a vocal style in which the artist speaks or chants along rhythmically with an instrumental or synthesized beat"
get_all_predictions(text_sentence)

Rapping <mask> as a vocal style in which the artist speaks or chants along rhythmically with an instrumental or synthesized beat


{'bart': 'is\ncan\noften\noriginated\ntypically',
 'bert': 'originated\nbegan\nrefers\nemerged\noccurs',
 'electra': 'acts\nserves\nworks\ncomes\ntranslates',
 'roberta': 'refers\ndefined\nmanifests\nstands\ndefines',
 'xlm': 'along\nappears\nserved\nworks\nbegan',
 'xlnet': 'ing\ns\nacts\nes\nis'}

In [182]:
text_sentence = "West Coast hip hop was <mask> by G-funk in the early-mid 1990s, while East Coast hip hop was dominated by jazz rap, alternative hip hop, and hardcore rap"
get_all_predictions(text_sentence)

West Coast hip hop was <mask> by G-funk in the early-mid 1990s, while East Coast hip hop was dominated by jazz rap, alternative hip hop, and hardcore rap


{'bart': 'dominated\nrevived\nboosted\nanchored\nstrongly',
 'bert': 'dominated\ninfluenced\ndominate\novershadowed\ninspired',
 'electra': 'dominated\ninfluenced\nfollowed\nled\ndriven',
 'roberta': 'dominated\nruled\nled\ninfluenced\ndefined',
 'xlm': 'inspired\ndeveloped\ncreated\nproduced\nfollowed',
 'xlnet': 'dominated\ninfluenced\nled\ndominant\nconsumed'}

In [192]:
text_sentence = "The prehistoric invention of shaped stone tools <mask> by the discovery of how to control fire increased sources of food"
get_all_predictions(text_sentence)

The prehistoric invention of shaped stone tools <mask> by the discovery of how to control fire increased sources of food


{'bart': 'accompanied\nfollowed\naided\nspurred\nboosted',
 'bert': 'followed\naccompanied\naided\ncoupled\nassisted',
 'electra': 'inspired\nfollowed\ncaused\ncreated\nled',
 'roberta': 'accompanied\nfollowed\npreceded\naugmented\nspurred',
 'xlm': 'followed\nled\ninspired\ncaused\nhelped',
 'xlnet': 'made\nby\nused\nreplaced\nintroduced'}

In [193]:
text_sentence = "The invention of the wheel <mask> humans to travel in and control their environment"
get_all_predictions(text_sentence)

The invention of the wheel <mask> humans to travel in and control their environment


{'bart': 'enabled\nallowed\nallows\nfreed\nenables',
 'bert': 'allowed\nenabled\nencouraged\npermitted\ntaught',
 'electra': 'allows\nenables\nrequires\nallowed\npermits',
 'roberta': 'enabled\nallowed\nempowered\ntaught\nfreed',
 'xlm': 'allowed\nallows\nenabled\nhelps\nhelped',
 'xlnet': 'human\nman\nhumans\nfor\nin'}

In [194]:
text_sentence = "The term technology comes from the Greek word techne, <mask> art and craft, and the word logos, meaning word and speech"
get_all_predictions(text_sentence)

The term technology comes from the Greek word techne, <mask> art and craft, and the word logos, meaning word and speech


{'bart': 'meaning\nrepresenting\nfor\nreferring\nrelated',
 'bert': 'meaning\nrepresenting\nindicating\ncombining\nmeans',
 'electra': 'meaning\nmeans\ngreek\nfor\nor',
 'roberta': 'meaning\nindicating\nfor\nmeaning\nor',
 'xlm': 'meaning\nfeeling\nmeans\nmean\nthe',
 'xlnet': 'meaning\nor\nfor\nword\nliterally'}

In [196]:
text_sentence = "8 million motherboards in the first quarter of 2015, which <mask> it to become the leading motherboard vendor"
get_all_predictions(text_sentence)

8 million motherboards in the first quarter of 2015, which <mask> it to become the leading motherboard vendor


{'bart': 'enabled\npropelled\nallowed\ncatapult\npushed',
 'bert': 'enabled\nallowed\npropelled\nhelped\nled',
 'electra': 'led\nallowed\nenabled\ncaused\nforced',
 'roberta': 'propelled\nenabled\nallowed\nled\npushed',
 'xlm': 'allowed\nenabled\nallows\ncaused\nhelped',
 'xlnet': 'it\nallowed\nmade\ngave\nled'}

In [198]:
text_sentence = " The earliest, widespread description of nanotechnology referred to the particular technological goal of precisely <mask> atoms and molecules for fabrication of macroscale products, also now referred to as molecular nanotechnology"
get_all_predictions(text_sentence)

 The earliest, widespread description of nanotechnology referred to the particular technological goal of precisely <mask> atoms and molecules for fabrication of macroscale products, also now referred to as molecular nanotechnology


{'bart': 'arranging\nmanipulating\ncombining\npacking\nmeasuring',
 'bert': 'exchanging\nconnecting\npreparing\nmanipulating\nseparating',
 'electra': 'using\nidentifying\npreparing\nproducing\ncreating',
 'roberta': 'arranging\nsplitting\nfolding\nmelting\ncombining',
 'xlm': 'the\ncertain\nvarious\nmany\nhuman',
 'xlnet': 'precisely\nonly\nspecifically\nexactly\nspecific'}

In [199]:
text_sentence = " A more generalized description of nanotechnology was subsequently <mask> by the National Nanotechnology Initiative, which defined nanotechnology as the manipulation of matter with at least one dimension sized from 1 to 100 nanometers"
get_all_predictions(text_sentence)

 A more generalized description of nanotechnology was subsequently <mask> by the National Nanotechnology Initiative, which defined nanotechnology as the manipulation of matter with at least one dimension sized from 1 to 100 nanometers


{'bart': 'published\nreleased\nadopted\nissued\nformulated',
 'bert': 'provided\ndeveloped\nadopted\ncreated\npublished',
 'electra': 'developed\npublished\nmade\nadopted\nprovided',
 'roberta': 'developed\nissued\ncreated\npublished\nproduced',
 'xlm': 'developed\nprovided\nprepared\noffered\npublished',
 'xlnet': 'provided\nadopted\nproposed\nfollowed\ndeveloped'}

In [200]:
text_sentence = " This definition reflects the fact that quantum mechanical effects are important at this quantum-realm scale, and so the definition <mask> from a particular technological goal to a research category inclusive of all types of research and technologies that deal with the special properties of matter which occur below the given size threshold"
get_all_predictions(text_sentence)

 This definition reflects the fact that quantum mechanical effects are important at this quantum-realm scale, and so the definition <mask> from a particular technological goal to a research category inclusive of all types of research and technologies that deal with the special properties of matter which occur below the given size threshold


{'bart': 'was\nshifted\nextends\nshifts\nis',
 'bert': 'ranges\nmoves\nextends\nspans\nshifts',
 'electra': 'ranges\nmoves\nshifts\ngoes\nextends',
 'roberta': 'shifts\ntransforms\nmoves\ntransitions\nevolves',
 'xlm': 'goes\nchanges\nextend\ndiffer\nrange',
 'xlnet': 'shifts\nextends\nmoves\nranges\nchanges'}

In [201]:
text_sentence = " Nanotechnology may be able to <mask> many new materials and devices with a vast range of applications, such as in nanomedicine, nanoelectronics, biomaterials energy production, and consumer products"
get_all_predictions(text_sentence)

 Nanotechnology may be able to <mask> many new materials and devices with a vast range of applications, such as in nanomedicine, nanoelectronics, biomaterials energy production, and consumer products


{'bart': 'produce\ndevelop\nsynthes\ncreate\nmanufacture',
 'bert': 'create\nproduce\ngenerate\ndevelop\nintroduce',
 'electra': 'provide\nproduce\ndeliver\ncreate\ndevelop',
 'roberta': 'produce\ncreate\ngenerate\nbuild\nengineer',
 'xlm': 'produce\ndevelop\nmanufacture\nintroduce\ncreate',
 'xlnet': 'produce\nprovide\ncreate\nto\ngenerate'}

In [202]:
text_sentence = "The original plan of the business was to build an object-oriented PC desktop environment <mask> a variety of applications"
get_all_predictions(text_sentence)

The original plan of the business was to build an object-oriented PC desktop environment <mask> a variety of applications


{'bart': 'for\nthat\ncapable\nto\nsuitable',
 'bert': 'for\nwith\ncontaining\nfeaturing\nsupporting',
 'electra': 'with\nfor\nusing\noffering\nincorporating',
 'roberta': 'for\nwith\nsupporting\naccommodating\ncontaining',
 'xlm': 'for\nwith\nin\nto\naround',
 'xlnet': 'with\na\nfor\nan\nsupporting'}

In [203]:
text_sentence = "<mask> me with your little candle"
get_all_predictions(text_sentence)

<mask> me with your little candle


{'bart': '<s>\n</s>\n""\nYou\n"\'',
 'bert': 'impress\npoke\nspray\nping\nhelp',
 'electra': 'bless\nlove\nhelp\nthank\ngrace',
 'roberta': 'light\nLight\nlight\nilluminate\nTouch',
 'xlm': 'Help\nJoin\nfeed\nentertain\nhelp',
 'xlnet': 'me\nwith\ngo\non\nbe'}

In [204]:
text_sentence = "Finally, my mother used to <mask> me to sleep reading portions of Hiawatha to me, especially: Wah-wah-taysee, little fire-fly"
get_all_predictions(text_sentence)

Finally, my mother used to <mask> me to sleep reading portions of Hiawatha to me, especially: Wah-wah-taysee, little fire-fly


{'bart': 'lull\nrock\nput\nnurse\ndrift',
 'bert': 'put\nrock\nlay\ntuck\ntalk',
 'electra': 'put\nteach\nbring\ncall\ntake',
 'roberta': 'rock\nput\nlull\ntalk\nsing',
 'xlm': 'allow\nget\nforce\nhelp\nput',
 'xlnet': 'to\nuse\ngive\nme\ntell'}

In [205]:
text_sentence = "Ere upon my bed I <mask> me"
get_all_predictions(text_sentence)

Ere upon my bed I <mask> me


{'bart': 'collapsed\ncalled\nsmelled\ncould\nheard',
 'bert': 'found\nlay\nlaid\ncalled\nkissed',
 'electra': 'found\nwoke\nsaw\nhad\nmet',
 'roberta': 'lay\nlaid\nburied\nsettled\nstuffed',
 'xlm': 'told\nlet\nmade\ngave\nfound',
 'xlnet': 'me\nI\nMe\nbe\nyou'}

In [206]:
text_sentence = "Ere in sleep I <mask> my eyelids!"
get_all_predictions(text_sentence)

Ere in sleep I <mask> my eyelids!


{'bart': 'closed\nsqueezed\nshut\nfl\nsw',
 'bert': 'fluttered\nshut\nclosed\nopened\ncracked',
 'electra': 'opened\nclosed\nlost\nopen\nlifted',
 'roberta': 'close\nclosed\nopen\nblink\nlift',
 'xlm': 'roll\nbalance\nrub\npump\nhurt',
 'xlnet': 'my\ngood\nyour\neven\nwell'}

In [208]:
text_sentence = "Where the heron, the Shuh-shuh-gah, <mask> among the reeds and rushes"
get_all_predictions(text_sentence)

Where the heron, the Shuh-shuh-gah, <mask> among the reeds and rushes


{'bart': 'swim\nsw\nro\nfl\nflies',
 'bert': 'floated\nlay\nswam\ndrifted\nstood',
 'electra': 'stands\nfalls\nis\nwalks\nwaits',
 'roberta': 'lives\nmoved\nnests\nlies\ndances',
 'xlm': 'sank\nruns\nspread\nswimming\nsitting',
 'xlnet': 'is\nwho\nin\neven\nlies'}

In [210]:
text_sentence = "Round about the Indian village <mask> the meadows and the corn-fields"
get_all_predictions(text_sentence)

Round about the Indian village <mask> the meadows and the corn-fields


{'bart': 'are\nwere\nspread\nin\nlay',
 'bert': 'were\nlay\nand\nspread\nwas',
 'electra': 'and\nare\nof\nwith\nin',
 'roberta': 'through\nin\nover\nto\nthru',
 'xlm': 'with\nin\nof\nbetween\naround',
 'xlnet': 'and\nby\nthe\nall\non'}

In [211]:
text_sentence = "And beyond them <mask> the forest"
get_all_predictions(text_sentence)

And beyond them <mask> the forest


{'bart': 'was\nran\nwere\nfloated\nspread',
 'bert': 'lay\nwas\nstretched\nspread\nstood',
 'electra': 'is\nare\nwas\nlies\nin',
 'roberta': 'in\nlay\nlies\nwas\ninto',
 'xlm': 'in\nis\nwas\ninto\non',
 'xlnet': 'the\nis\nall\nare\nwas'}

In [212]:
text_sentence = "<mask> the groves of singing pine-tree"
get_all_predictions(text_sentence)

<mask> the groves of singing pine-tree


{'bart': '</s>\n<s>\nThe\nI\n�',
 'bert': 'amid\namong\nin\namidst\namongst',
 'electra': 'in\nsee\nand\nfrom\nlike',
 'roberta': 'pan\nin\ning\nCapture\nIn',
 'xlm': 'Through\nOver\nin\nInside\nIn',
 'xlnet': 'the\nand\nin\nall\non'}

In [213]:
text_sentence = "You could <mask> them through the valley"
get_all_predictions(text_sentence)

You could <mask> them through the valley


{'bart': 'follow\nsee\nwave\nrun\nride',
 'bert': 'follow\nchase\ntrack\nlead\nride',
 'electra': 'take\nsee\npass\ndrive\nfollow',
 'roberta': 'see\ntrace\ntrail\ncarry\nchase',
 'xlm': 'push\ntake\ndrive\nlead\ncarry',
 'xlnet': 'them\nlead\nmake\nit\nher'}

In [215]:
text_sentence = "Love the wind among the branches, and the rain-shower and the snow-storm, and the <mask> of great rivers"
get_all_predictions(text_sentence)

Love the wind among the branches, and the rain-shower and the snow-storm, and the <mask> of great rivers


{'bart': 'rushing\nswollen\nswelling\nroar\nroaring',
 'bert': 'flow\nrush\nflowing\nrushing\ntrickle',
 'electra': 'flow\nflood\nflooding\npresence\nroar',
 'roberta': 'flow\nswelling\nrushing\noverflow\nrushes',
 'xlm': 'flow\ngulf\nformation\nflood\nwater',
 'xlnet': 'wind\nthe\nall\nhow\nwinds'}

In [216]:
text_sentence = "And the thunder in the mountains, whose innumerable echoes <mask> like eagles in their eyries"
get_all_predictions(text_sentence)

And the thunder in the mountains, whose innumerable echoes <mask> like eagles in their eyries


{'bart': 'echoed\nr\nfloated\ncrack\ncl',
 'bert': 'rang\nfluttered\nlay\nbuzzed\nhung',
 'electra': 'sound\nare\nsounded\nwere\nspread',
 'roberta': 'danced\nshone\nlay\ndance\nechoed',
 'xlm': 'were\nare\nspread\nfeel\nsound',
 'xlnet': 'echo\nechoes\nechoed\nlike\nthunder'}

In [217]:
text_sentence = "<mask> in tones so plain and childlike"
get_all_predictions(text_sentence)

<mask> in tones so plain and childlike


{'bart': '<s>\n</s>\nThe\nHe\n"\'',
 'bert': 'spoke\nspeaking\nsinging\nspoken\ntalking',
 'electra': 'speaking\nbut\nwords\ndressed\neven',
 'roberta': 'lied\ncommunicated\nbargaining\npan\ntalk',
 'xlm': '</s>\ncream\nMeet\n<pad>\nສໍານວນ',
 'xlnet': 'in\nas\nforth\nperson\nsuch'}

In [218]:
text_sentence = "Scarcely can the ear <mask> whether they are sung or spoken"
get_all_predictions(text_sentence)

Scarcely can the ear <mask> whether they are sung or spoken


{'bart': 'hear\ndetect\ndiscern\nperceive\npenetrate',
 'bert': 'determine\nidentify\ntell\ndetect\ndistinguish',
 'electra': 'know\ntell\ndetermine\nhear\ndecide',
 'roberta': 'discern\ndistinguish\nhear\ndetermine\nperceive',
 'xlm': 's\ndetermine\nwonder\nhear\nunderstand',
 'xlnet': 'know\nhead\ncare\ndo\nscan'}

In [219]:
text_sentence = "Ye whose hearts are fresh and simple, who <mask> faith in God and Nature"
get_all_predictions(text_sentence)

Ye whose hearts are fresh and simple, who <mask> faith in God and Nature


{'bart': 'have\nprofess\nplace\nput\nbelieve',
 'bert': 'have\nhold\nkeep\nshow\nhad',
 'electra': 'have\nlack\nshare\nshow\nlose',
 'roberta': 'have\nshare\nprofess\nkeep\ntake',
 'xlm': 'has\nhave\nexercise\nshare\nhad',
 'xlnet': 'put\nhave\nmake\nsay\nlack'}

In [220]:
text_sentence = "For the good they comprehend not that the feeble hands and helpless, <mask> blindly in the darkness"
get_all_predictions(text_sentence)

For the good they comprehend not that the feeble hands and helpless, <mask> blindly in the darkness


{'bart': 'frightened\nterrified\nhelpless\nblind\nelderly',
 'bert': 'wandered\nran\nswam\nstruggled\nstood',
 'electra': 'but\nwalking\ngoing\nand\nnot',
 'roberta': 'wander\nwalk\nlay\nroam\nhunt',
 'xlm': 'sitting\nwere\nsit\nare\nsleeping',
 'xlnet': 'blind\nstand\nand\nare\nso'}

In [221]:
text_sentence = "<mask> God's right hand in that darkness"
get_all_predictions(text_sentence)

<mask> God's right hand in that darkness


{'bart': '<s>\nI\n</s>\nThe\nWe',
 'bert': 'lay\nplaced\nheld\nput\nplace',
 'electra': 'put\nsee\nand\nfind\nputting',
 'roberta': 'Spot\npan\n<pad>\nhus\nbargaining',
 'xlm': '</s>\nweather\nສໍານວນ\nEnjoy\nfeel',
 'xlnet': 'hand\nGod\n’\ns\nand'}

In [222]:
text_sentence = "Through the green lanes of the country, where the tangled barberry-bushes <mask> their tufts of crimson berries"
get_all_predictions(text_sentence)

Through the green lanes of the country, where the tangled barberry-bushes <mask> their tufts of crimson berries


{'bart': 'cl\nswayed\nhung\nspr\nspread',
 'bert': 'shed\ngrew\nspread\npoked\nfluttered',
 'electra': 'spread\ngrow\nshed\ngrew\ncut',
 'roberta': 'spread\nshow\ngrow\nshed\nlay',
 'xlm': 'spread\nshared\nwith\nmake\nshare',
 'xlnet': 'produce\nhave\ntheir\nand\nmake'}

In [224]:
text_sentence = "Over stone walls gray with mosses, <mask> by some neglected graveyard"
get_all_predictions(text_sentence)

Over stone walls gray with mosses, <mask> by some neglected graveyard


{'bart': 'the\nit\nthey\nsurrounded\nflanked',
 'bert': 'surrounded\nbordered\novergrown\noccupied\nbounded',
 'electra': 'surrounded\nflanked\ncovered\nfollowed\nobscured',
 'roberta': 'surrounded\nframed\nburied\nswallowed\nlined',
 'xlm': 'followed\nserved\ncaused\nled\nshared',
 'xlnet': 'probably\napparently\nlikely\npossibly\npresumably'}

In [227]:
text_sentence = "He the Master of Life, <mask> on the red crags of the quarry"
get_all_predictions(text_sentence)

He the Master of Life, <mask> on the red crags of the quarry


{'bart': 'perched\nhe\nrecl\nfloated\nwalked',
 'bert': 'perched\nsitting\nliving\nlay\nresting',
 'electra': 'laying\nstanding\nsitting\nliving\nworking',
 'roberta': 'lay\nlived\nlies\nwalked\nlives',
 'xlm': 'sitting\nsleeping\nworking\ndecided\n</s>',
 'xlnet': 'while\non\nand\nthat\nwho'}

In [229]:
text_sentence = "Stood erect, and <mask> the nations, called the tribes of men together"
get_all_predictions(text_sentence)

Stood erect, and <mask> the nations, called the tribes of men together


{'bart': 'together\nto\nbefore\naddressed\naddressing',
 'bert': 'surveyed\nassembled\namong\namid\nall',
 'electra': 'all\nin\nof\nthen\ncalled',
 'roberta': 'united\nbefore\nshook\ncalled\nspread',
 'xlm': 'all\namong\nmany\nmeeting\nheard',
 'xlnet': 'the\nall\nthen\nthat\nin'}

In [230]:
text_sentence = "From his footprints flowed a river, <mask> into the light of morning"
get_all_predictions(text_sentence)

From his footprints flowed a river, <mask> into the light of morning


{'bart': 'which\nflowing\nrushing\nswelling\nbursting',
 'bert': 'flowing\nemerging\npouring\nstreaming\nbursting',
 'electra': 'and\nthen\nflowing\neven\ndeep',
 'roberta': 'flowing\nstreaming\nspilling\nbursting\nsparkling',
 'xlm': 'running\ncoming\nwell\nbuilt\nback',
 'xlnet': 'and\ndown\nthe\never\nfirst'}

In [231]:
text_sentence = "And the Spirit, <mask> earthward with his finger on the meadow"
get_all_predictions(text_sentence)

And the Spirit, <mask> earthward with his finger on the meadow


{'bart': 'walking\nwaving\npointing\nlooking\nhe',
 'bert': 'pointing\npointed\ngesturing\npoints\ngestured',
 'electra': 'standing\nlaying\nsitting\nwalking\nlanding',
 'roberta': 'moved\nfloated\nwalked\nascended\ndrifted',
 'xlm': 'went\ntravel\nwalked\nthe\nfell',
 'xlnet': 'right\nout\nimmediately\non\nborn'}

In [233]:
text_sentence = "The genre <mask> in the 1960s and 1970s, when new musical instruments were being introduced to a wider market, such as the synthesizer"
get_all_predictions(text_sentence)

The genre <mask> in the 1960s and 1970s, when new musical instruments were being introduced to a wider market, such as the synthesizer


{'bart': 'flourished\nbo\nwas\ngrew\nexploded',
 'bert': 'emerged\noriginated\npeaked\nflourished\narose',
 'electra': 'began\nchanged\nexpanded\nevolved\ndeveloped',
 'roberta': 'emerged\nspread\nmatured\ndeveloped\nexploded',
 'xlm': 'began\ndeveloped\nstarted\nchanged\narrived',
 'xlnet': 'began\nstarted\nemerged\nwas\noriginated'}

In [236]:
text_sentence = "Due to its relatively open style, ambient music often <mask> influences from many other genres, ranging from classical, avant-garde music, folk, jazz, and world music, amongst others"
get_all_predictions(text_sentence)

Due to its relatively open style, ambient music often <mask> influences from many other genres, ranging from classical, avant-garde music, folk, jazz, and world music, amongst others


{'bart': 'draws\nincorporates\nhas\nblends\ncombines',
 'bert': 'draws\nincorporates\ntakes\nreflects\nreceives',
 'electra': 'has\ndraws\nincorporates\nattracts\ninfluences',
 'roberta': 'draws\nreceives\nattracts\ntakes\npulls',
 'xlm': 'has\nhad\nincludes\ngets\ndraws',
 'xlnet': 'influences\ntakes\nfeatures\nforms\nheavily'}

In [237]:
text_sentence = "The genre <mask> its name from the Atlanta slang word trap, which refers to a place in which drugs are sold illegally"
get_all_predictions(text_sentence)

The genre <mask> its name from the Atlanta slang word trap, which refers to a place in which drugs are sold illegally


{'bart': 'derives\ngets\ntakes\nderived\ngot',
 'bert': 'derives\ntakes\ndraws\nderived\ngets',
 'electra': 'derives\ntakes\ndraws\ngets\ntook',
 'roberta': 'takes\nderives\ndraws\ntook\ngets',
 'xlm': 'takes\ngot\ngets\ntook\nhas',
 'xlnet': 'takes\nits\nis\ngets\nitself'}

In [238]:
text_sentence = " It has <mask> the music of many pop and R&B artists, such as Ariana Grande, Beyoncé, Miley Cyrus, Rihanna and more"
get_all_predictions(text_sentence)

 It has <mask> the music of many pop and R&B artists, such as Ariana Grande, Beyoncé, Miley Cyrus, Rihanna and more


{'bart': 'influenced\ninspired\nboosted\nbecome\nenriched',
 'bert': 'influenced\ninspired\nrecorded\nplayed\nfeatured',
 'electra': 'been\ninfluenced\ninspired\nbecome\nproduced',
 'roberta': 'influenced\nenriched\nchanged\nrevived\ntransformed',
 'xlm': 'inspired\nbecome\nchanged\nbeen\nmade',
 'xlnet': 'made\nthe\nbeen\nhad\ninfluenced'}

In [239]:
text_sentence = " In 2019, the trap-inspired country/rap crossover Old Town Road by Lil Nas X (featuring Billy Ray Cyrus) <mask> the record for spending the most weeks (19 weeks) at No"
get_all_predictions(text_sentence)

 In 2019, the trap-inspired country/rap crossover Old Town Road by Lil Nas X (featuring Billy Ray Cyrus) <mask> the record for spending the most weeks (19 weeks) at No


{'bart': 'holds\nbroke\nwon\nshattered\nset',
 'bert': 'broke\nholds\nset\ntied\nheld',
 'electra': 'set\nsets\nbroke\nholds\nheld',
 'roberta': 'broke\ntook\nshattered\nsmashed\ntied',
 'xlm': 'set\nestablished\nhas\nclosed\nwon',
 'xlnet': 'holds\nbreaks\nsets\nhold\nrecords'}

In [240]:
text_sentence = "The film <mask> an adaptation of the 1959 stage musical of the same name, composed by Richard Rodgers with lyrics by Oscar Hammerstein II"
get_all_predictions(text_sentence)

The film <mask> an adaptation of the 1959 stage musical of the same name, composed by Richard Rodgers with lyrics by Oscar Hammerstein II


{'bart': 'is\nwas\n""\nversion\nadaptation',
 'bert': 'is\nwas\nfeatures\ncontains\nincludes',
 'electra': 'is\nwas\nfeatures\nfollows\nincludes',
 'roberta': 'was\nis\nmarks\nmarked\nself',
 'xlm': 'is\nwas\nhas\nprovides\noffers',
 'xlnet': 'is\nwas\nfilm\nan\nradio'}

In [241]:
text_sentence = "After <mask> love and music into the lives of the family, she marries the officer and, together with the children, finds a way to survive the loss of their homeland to the Nazis"
get_all_predictions(text_sentence)

After <mask> love and music into the lives of the family, she marries the officer and, together with the children, finds a way to survive the loss of their homeland to the Nazis


{'bart': 'introducing\ninjecting\npouring\nshe\nbringing',
 'bert': 'introducing\npouring\nincorporating\nintegrating\nject',
 'electra': 'pouring\nbringing\nputting\nincorporating\nintroducing',
 'roberta': 'introducing\nbringing\ninjecting\nbreathing\nabsorbing',
 'xlm': 'bringing\nadding\nthe\nincreasing\nputting',
 'xlnet': 'bringing\nintroducing\ninviting\nallowing\npouring'}

In [244]:
text_sentence = "EMI\'s locations in the United Kingdom, the United States, and Canada were all <mask> to repay debt, but the primary head office located outside those countries is still functional"
get_all_predictions(text_sentence)

EMI's locations in the United Kingdom, the United States, and Canada were all <mask> to repay debt, but the primary head office located outside those countries is still functional


{'bart': 'raided\nseized\nmortg\nsold\nclosed',
 'bert': 'used\nunable\nclosed\nstruggling\nable',
 'electra': 'unable\ndesigned\nable\nsupposed\nused',
 'roberta': 'drained\nclosed\nused\nsold\ndissolved',
 'xlm': 'able\nexpected\ntrying\ndesigned\nsupposed',
 'xlnet': 'all\ndue\ndeemed\nforced\nalso'}

In [250]:
text_sentence = "With the bark of the red willow, <mask> upon the neighboring forest"
get_all_predictions(text_sentence)

With the bark of the red willow, <mask> upon the neighboring forest


{'bart': 'it\nthe\nsunlight\nlight\nthey',
 'bert': 'descended\nfell\nlay\ngazed\nconverge',
 'electra': 'look\nfall\ncome\ndescend\nreturn',
 'roberta': 'descended\nfell\ndescend\nlay\nspread',
 'xlm': 'rain\nborder\ngaze\nsank\nfell',
 'xlnet': 'and\nupon\nthat\non\n<eop>'}

In [251]:
text_sentence = "Till in flame they <mask> and kindled"
get_all_predictions(text_sentence)

Till in flame they <mask> and kindled


{'bart': 'were\nflared\ncaught\nspread\nflashed',
 'bert': 'burned\nheated\nspread\nburnt\nblazed',
 'electra': 'stirred\nburned\npoured\nwashed\ndried',
 'roberta': 'burned\nignited\nexploded\ndanced\nmelted',
 'xlm': 'laugh\nrose\nspoke\nstood\nposed',
 'xlnet': 'and\nwere\nwent\nall\nwas'}

In [254]:
text_sentence = "Then a snow-white cloud unfolding, Like the tree-tops of the forest, ever rising, till it <mask> the top of heaven"
get_all_predictions(text_sentence)

Then a snow-white cloud unfolding, Like the tree-tops of the forest, ever rising, till it <mask> the top of heaven


{'bart': 'reached\nreaches\nfloated\nwas\nlanded',
 'bert': 'reached\ncrested\nmet\ntouched\ntopped',
 'electra': 'reaches\nreached\nhits\nhit\nmeets',
 'roberta': 'reached\nreaches\ntouched\ntouches\nascended',
 'xlm': 'reached\nreach\nat\nis\nwas',
 'xlnet': 'was\nbecame\nbecome\nthe\nreach'}

In [276]:
text_sentence = "Over them he strecthed his right hand, to <mask> their stubborn natures"
get_all_predictions(text_sentence)

Over them he strecthed his right hand, to <mask> their stubborn natures


{'bart': 'def\ntemper\ncontrol\nquiet\ncalm',
 'bert': 'emphasize\ntame\ndemonstrate\nmatch\ncombat',
 'electra': 'conceal\nhide\nremove\nprotect\novercome',
 'roberta': 'bind\ncool\nappease\nstrengthen\ntame',
 'xlm': 'balance\ncontrol\nexplain\nalter\nescape',
 'xlnet': 'their\nto\nthey\nthem\nthe'}

In [274]:
text_sentence = "The examples of Africa, Indonesia, and the Middle East show how Western pop music styles can <mask> with local musical traditions to create new hybrid styles"
get_all_predictions(text_sentence)

The examples of Africa, Indonesia, and the Middle East show how Western pop music styles can <mask> with local musical traditions to create new hybrid styles


{'bart': 'merge\nblend\nco\nbe\nfuse',
 'bert': 'fuse\nmerge\nmix\ncombine\nblend',
 'electra': 'combine\nblend\nmerge\nmix\ncompete',
 'roberta': 'fuse\nmerge\nblend\ncombine\nmix',
 'xlm': 'mix\nblend\ninteract\ncompete\nmerge',
 'xlnet': 'work\nplay\npartner\nbreak\nconnect'}

In [277]:
text_sentence = "To <mask> their thirst and fever, by the shadow of his right hand"
get_all_predictions(text_sentence)

To <mask> their thirst and fever, by the shadow of his right hand


{'bart': 'qu\nst\nward\nextingu\ncontrol',
 'bert': 'water\nfeed\ncure\nease\nsatisfy',
 'electra': 'relieve\ncure\nsatisfy\novercome\ncalm',
 'roberta': 'cool\nease\ndrain\ndrown\ncure',
 'xlm': 'control\nreduce\nfight\nlimit\ncalm',
 'xlnet': 'To\nTo\nto\nto\nDo'}

In [279]:
text_sentence = "As the sound of far-off waters, <mask> into deep abysses"
get_all_predictions(text_sentence)

As the sound of far-off waters, <mask> into deep abysses


{'bart': 'it\nthey\nplunged\nplung\nthe',
 'bert': 'descending\nplunging\ndrifted\nsank\nsinking',
 'electra': 'plunging\nrising\nfalling\nplunged\nturning',
 'roberta': 'drains\nsinks\nmelts\nsinking\ndips',
 'xlm': 'spread\nflood\nsank\ndisappear\nfell',
 'xlnet': 'out\non\ngo\nin\nso'}

In [282]:
text_sentence = "Bathe now in the <mask> before you"
get_all_predictions(text_sentence)

Bathe now in the <mask> before you


{'bart': 'lake\nriver\npool\nwater\nsea',
 'bert': 'water\ntub\nbath\nsteam\nsink',
 'electra': 'room\nplace\nwater\nmoment\nsun',
 'roberta': 'sea\nwater\nair\npool\nroom',
 'xlm': 'water\nroom\nair\nbath\npool',
 'xlnet': 'place\nin\nplaces\ncity\nthe'}

In [283]:
text_sentence = "Wash the <mask> from your faces"
get_all_predictions(text_sentence)

Wash the <mask> from your faces


{'bart': 'tears\nsweat\ndirt\nmakeup\ndro',
 'bert': 'sweat\nsteam\ncolor\nblood\ntears',
 'electra': 'dust\nsand\nblood\nwater\nhair',
 'roberta': 'sweat\ncolor\nblood\ncolour\ndirt',
 'xlm': 'oil\nwater\nsnow\nair\nblood',
 'xlnet': 'the\nin\nor\nfar\nway'}

In [288]:
text_sentence = "Bury your <mask> and your weapons"
get_all_predictions(text_sentence)

Bury your <mask> and your weapons


{'bart': 'books\nanger\nshoes\nmoney\nweapons',
 'bert': 'clothes\nequipment\nbody\nbodies\nremains',
 'electra': 'hands\nbones\nbody\nhead\nbodies',
 'roberta': 'money\nbooks\nconscience\npossessions\nbody',
 'xlm': 'hair\nhands\narms\nclothes\nhead',
 'xlnet': 'your\nany\nyou\nall\nown'}

In [292]:
text_sentence = "Take the <mask> that grow beside you"
get_all_predictions(text_sentence)

Take the <mask> that grow beside you


{'bart': 'flowers\nwild\ntrees\nleaves\nfruits',
 'bert': 'flowers\nvines\nroses\nseeds\nplants',
 'electra': 'things\nones\ntrees\nhands\nflowers',
 'roberta': 'leaves\nplants\nflowers\nmushrooms\nroses',
 'xlm': 'flowers\nvegetables\ntree\nfruits\nparasites',
 'xlnet': 'the\nthat\none\nyou\nwho'}

In [293]:
text_sentence = "Deck them with your brightest <mask>"
get_all_predictions(text_sentence)

Deck them with your brightest <mask>


{'bart': 'lights\npenn\ncolors\nbulbs\ncolours',
 'bert': 'lights\nlight\nsmile\nideas\nflare',
 'electra': 'star\nstars\nones\neye\ncolors',
 'roberta': 'star\nlight\nred\ncolors\nstars',
 'xlm': 'colors\nenergy\nlight\neyes\nimagination',
 'xlnet': '<eop>\nand\nagain\nI'}

In [297]:
text_sentence = "Then upon the ground the warriors threw their cloaks and shirts of <mask>"
get_all_predictions(text_sentence)

Then upon the ground the warriors threw their cloaks and shirts of <mask>


{'bart': 'leather\ndarkness\nvictory\nblack\ncrimson',
 'bert': 'mail\nmourning\npaint\nwar\nsweat',
 'electra': 'armor\ngold\nclothing\narmour\narms',
 'roberta': 'cloth\nwhite\nred\nhonour\nfabric',
 'xlm': 'men\nwar\ncolor\nit\nrain',
 'xlnet': 'of\nthe\nwhich\nthat\nthey'}

In [298]:
text_sentence = "Threw their <mask> and their war-gear"
get_all_predictions(text_sentence)

Threw their <mask> and their war-gear


{'bart': 'weapons\nswords\ncoats\nguns\nbelts',
 'bert': 'weapons\naxes\nspears\ntents\nswords',
 'electra': 'weapons\nguns\narms\ntime\nwar',
 'roberta': 'hats\nbooks\ncoats\nweapons\nclothes',
 'xlm': 'family\nmother\nfriends\nwife\narms',
 'xlnet': 'war\ns\nand\nown\ntheir'}

In [302]:
text_sentence = "From the river came the warriors, clean and washed from all their <mask>"
get_all_predictions(text_sentence)

From the river came the warriors, clean and washed from all their <mask>


{'bart': 'sins\nanger\nexcess\nblood\ndrinking',
 'bert': 'wounds\ninjuries\nclothes\nbodies\npossessions',
 'electra': 'wounds\nsuffering\ninjuries\nsins\nbodies',
 'roberta': 'wounds\nbattles\nbodies\nwear\nscars',
 'xlm': 'clothes\nhair\nhands\narms\nfeet',
 'xlnet': 'their\nrespective\nprevious\nin\na'}

In [303]:
text_sentence = "On the banks their <mask> they buried"
get_all_predictions(text_sentence)

On the banks their <mask> they buried


{'bart': 'hearts\nbodies\ncorpses\nhorses\ndead',
 'bert': 'horses\nboats\nfriends\nquarry\ngraves',
 'electra': 'banks\nnames\ngraves\nchildren\nlives',
 'roberta': 'books\nmoney\naccounts\ndeposits\nfunds',
 'xlm': 'mother\nchildren\nlife\nfamily\nhorse',
 'xlnet': 'they\nto\nthe\nhe\nand'}

In [304]:
text_sentence = "Buried all their warlike <mask>"
get_all_predictions(text_sentence)

Buried all their warlike <mask>


{'bart': 'beliefs\nambitions\nideas\nfeelings\nways',
 'bert': 'enemies\ncomrades\nsecrets\nheroes\nsoldiers',
 'electra': 'bodies\nwounds\nlives\nfaces\npieces',
 'roberta': 'hearts\nblood\nbodies\nshells\nfaces',
 'xlm': 's\nly\nness\ncharacters\nfriends',
 'xlnet': 'nature\nspirit\nintelligence\nactivities\nbehavior'}

In [308]:
text_sentence = "Till the hot breath of his nostrils warmed the <mask> of Mudjekeewis"
get_all_predictions(text_sentence)

Till the hot breath of his nostrils warmed the <mask> of Mudjekeewis


{'bart': 'face\nnight\ncheeks\nhair\nback',
 'bert': 'skin\nsweat\nblood\nsurface\nflesh',
 'electra': 'smell\nnostrils\nscent\neyes\nbreath',
 'roberta': 'hearts\nair\nskin\nbones\nheart',
 'xlm': 'skin\nblood\nmouth\nbody\nface',
 'xlnet': 'the\nneed\nway\npart\nheart'}

In [309]:
text_sentence = "The heavy breathing warmed the <mask> of Mudjekeewis"
get_all_predictions(text_sentence)

The heavy breathing warmed the <mask> of Mudjekeewis


{'bart': 'cabin\nbed\nair\noffice\nroom',
 'bert': 'skin\nsweat\nflesh\nblood\nface',
 'electra': 'spirits\nminds\nhearts\nheart\nspirit',
 'roberta': 'hearts\nair\nheart\nskin\nbones',
 'xlm': 'skin\nbody\nblood\nheart\nface',
 'xlnet': 'the\nof\nway\nbody\nout'}

In [311]:
text_sentence = "Then he <mask> his war-club"
get_all_predictions(text_sentence)

Then he <mask> his war-club


{'bart': 'locked\nfet\nmarched\nstamped\ncl',
 'bert': 'raised\nopened\ngrabbed\nunleashed\nactivated',
 'electra': 'started\njoined\nleft\nformed\nfounded',
 'roberta': 'started\njoined\nformed\nexpanded\nopened',
 'xlm': 'began\nlost\nleft\nstarted\nfound',
 'xlnet': 'his\nhe\nhad\nhas\nis'}

In [312]:
text_sentence = "Right between the <mask> he smote him"
get_all_predictions(text_sentence)

Right between the <mask> he smote him


{'bart': 'soldiers\npoliceman\nguards\nbusinessman\nguard',
 'bert': 'legs\neyes\nears\nfeet\nribs',
 'electra': 'two\nlines\nmen\nwords\ndays',
 'roberta': 'teeth\nsheets\nscars\nblows\nlines',
 'xlm': 'lines\nwrinkles\nwords\ntimes\ntime',
 'xlnet': 'the\ntwo\nand\nwhere\nto'}

In [313]:
text_sentence = "But his <mask> beneath him trembled"
get_all_predictions(text_sentence)

But his <mask> beneath him trembled


{'bart': 'knees\nlegs\nchair\nstomach\nmuscles',
 'bert': 'legs\nknees\nmuscles\nbody\nfeet',
 'electra': 'body\nfeet\nskin\nface\narms',
 'roberta': 'muscles\nbody\nhands\nskin\nfeet',
 'xlm': 'feet\nblood\nfriends\narms\nlife',
 'xlnet': 's\non\nbeneath\n’\nhis'}

In [314]:
text_sentence = "Bear! you know our <mask> are hostile"
get_all_predictions(text_sentence)

Bear! you know our <mask> are hostile


{'bart': 'neighbors\nhearts\nrules\npolitics\nresidents',
 'bert': 'neighbors\npeople\nenemies\nneighbours\nanimals',
 'electra': 'enemies\npeople\nforces\nneighbors\ntroops',
 'roberta': 'hearts\nintentions\npeople\nfeelings\nrooms',
 'xlm': 'customers\nguests\ncharacters\nvisitors\npeople',
 'xlnet': 'are\ns\nis\nwe\npeople'}

In [316]:
text_sentence = "You go hiding in the <mask>!"
get_all_predictions(text_sentence)

You go hiding in the <mask>!


{'bart': 'bushes\nhouse\nbathroom\nbasement\nwoods',
 'bert': 'woods\nbushes\nshadows\nattic\ncloset',
 'electra': 'city\nwoods\ndesert\ncave\ndark',
 'roberta': 'woods\nforest\nbushes\nmountains\nsnow',
 'xlm': 'water\nriver\nair\nrain\nsnow',
 'xlnet': 'the\nThe\nin\nto\na'}

In [317]:
text_sentence = "For himself he kept the West-Wind, gave the <mask> to his children"
get_all_predictions(text_sentence)

For himself he kept the West-Wind, gave the <mask> to his children


{'bart': 'horses\nkeys\nlamp\nhouse\nland',
 'bert': 'wind\nwinds\nsun\nwater\nair',
 'electra': 'wind\nwest\npower\nland\nwater',
 'roberta': 'plant\nwind\nbird\nWind\nbook',
 'xlm': 'water\nmilk\nroom\noil\nbait',
 'xlnet': 'to\nover\nit\nthe\nin'}

In [321]:
text_sentence = "With his flattering words he <mask> her"
get_all_predictions(text_sentence)

With his flattering words he <mask> her


{'bart': 'woo\nent\nsed\nchar\next',
 'bert': 'kissed\nsilenced\naddressed\nteased\nshowed',
 'electra': 'kissed\nbeat\ntouched\ngave\nkilled',
 'roberta': 'lured\nwarmed\nmelted\nmoved\nenchanted',
 'xlm': 'surprised\nhelped\ndisappointed\ntold\nimpressed',
 'xlnet': 'we\nhe\nus\na\nme'}

In [322]:
text_sentence = "Folded in his robes of <mask>"
get_all_predictions(text_sentence)

Folded in his robes of <mask>


{'bart': 'silk\nleather\nrighteousness\nhonor\ngold',
 'bert': 'state\noffice\ngovernment\nhonor\nmourning',
 'electra': 'clothing\nhonor\ngrace\nservice\ndress',
 'roberta': 'honour\nhonor\nrank\noffice\nmourning',
 'xlm': 'summer\nwinter\nhair\nlife\nrain',
 'xlnet': 'the\nof\nto\nand\na'}

In [323]:
text_sentence = "Till into a star he <mask> her"
get_all_predictions(text_sentence)

Till into a star he <mask> her


{'bart': 'flashed\nkissed\nstared\npursued\nmounted',
 'bert': 'floated\npainted\nkissed\ntransformed\nturned',
 'electra': 'sent\ngave\nbrought\nmade\ntook',
 'roberta': 'transformed\nswallowed\nchanged\nkissed\ndrained',
 'xlm': 'caught\nlost\nsaw\nleft\nmade',
 'xlnet': 'he\nhis\nhim\ns\na'}

In [328]:
text_sentence = "Still he did not <mask> his laughing"
get_all_predictions(text_sentence)

Still he did not <mask> his laughing


{'bart': 'hold\nhide\nstop\ncontrol\ncease',
 'bert': 'stop\nrestrain\nhide\ncease\nsuppress',
 'electra': 'like\nstop\nenjoy\nmiss\nshow',
 'roberta': 'stop\ncease\nrestrain\nhide\nlose',
 'xlm': 'stop\nreduce\nlimit\nalter\ncut',
 'xlnet': 'his\nhave\nmake\nher\nor'}

In [331]:
text_sentence = "Drops of sweat <mask> fast and heavy"
get_all_predictions(text_sentence)

Drops of sweat <mask> fast and heavy


{'bart': 'ran\nstreamed\ndri\nwere\nflowed',
 'bert': 'fell\nflowed\ndropped\ndripped\nflew',
 'electra': 'are\nwere\nwent\ncame\ngo',
 'roberta': 'fall\nair\nare\nflow\nmove',
 'xlm': 'are\nwere\nis\nwas\ngo',
 'xlnet': 'sweat\nvery\nhot\nfast\nmade'}

In [332]:
text_sentence = "Till at last he rose <mask>"
get_all_predictions(text_sentence)

Till at last he rose <mask>


{'bart': 'to\nfrom\nagain\nabove\non',
 'bert': 'again\nslowly\nup\nslightly\nrapidly',
 'electra': 'again\nup\nhigh\nwell\nhigher',
 'roberta': 'up\nagain\ndecisively\nquickly\nhimself',
 'xlm': 's\nup\naway\nthrough\nbefore',
 'xlnet': 'rose\nagain\nup\ntoo\nto'}

In [336]:
text_sentence = "Till his panting breath grew <mask>"
get_all_predictions(text_sentence)

Till his panting breath grew <mask>


{'bart': 'audible\nshallow\nlab\nfast\nfoul',
 'bert': 'ragged\nheavy\nheavier\nerratic\nshort',
 'electra': 'stronger\nlouder\ndeeper\nheavier\nharder',
 'roberta': 'stronger\nlonger\nlouder\nlong\nheavier',
 'xlm': 'up\nhigh\nmore\nwide\nair',
 'xlnet': 'again\nin\non\nup\nis'}

In [344]:
text_sentence = "But he was too fat and lazy to <mask> himself and woo her"
get_all_predictions(text_sentence)

But he was too fat and lazy to <mask> himself and woo her


{'bart': 'dress\nclot\ncompose\nportray\ncontrol',
 'bert': 'tame\nexcuse\nshave\ndress\nstroke',
 'electra': 'kill\ndefend\nsell\nlove\npush',
 'roberta': 'dress\ncompose\nlift\ndefend\nstretch',
 'xlm': 'defend\nkill\nfeed\nhelp\nhurt',
 'xlnet': 'to\npress\nuse\nindulge\nher'}

In [347]:
text_sentence = "In the eastern sky, the rainbow <mask>, What is that, Nokomis?"
get_all_predictions(text_sentence)

In the eastern sky, the rainbow <mask>, What is that, Nokomis?


{'bart': 'flashed\nbl\nfloated\ndanced\ngl',
 'bert': 'glowed\nfloated\nsang\nflashed\nappeared',
 'electra': 'says\nis\nasks\nsounds\nwhispers',
 'roberta': 'flashed\nsaid\nflashes\nspoke\ndanced',
 'xlm': 's\n</s>\nrose\nappeared\nappears',
 'xlnet': 'rainbow\nlow\nline\nsky\nhigh'}